In [ ]:
import os
import numpy as np
import pandas
import re
import urllib
from bs4 import BeautifulSoup as bs
from datetime import datetime as dt

debug = 2
print('ok')

In [ ]:
os.chdir('/home/idies/workspace/Storage/raddick/raddick_folksonomy/data/secondpass')
music_df = pandas.read_csv('cleaned_audio_data.csv', index_col='ID', encoding='utf-8')
#music_df.loc['a95d77f177fd529791c6486ee6c177967f4d235c']

music_df = music_df.rename(columns={'Source': 'source', 'Sample File Name': 'filename'})

# Everything after the final period is the filetype
music_df['filetype'] = music_df['filename'].apply(lambda x: x.lower().split('.')[-1])
# Everything before the final period is the stem
music_df['stem'] = music_df['filename'].apply(lambda x: x[0:len(x)-len(x.split('.')[-1])-1])

# Get ready to score
music_df['score'] = -9999

print('music_df has {0:,.0f} total rows.'.format(len(music_df)))
music_df_bk = music_df
music_df.head(4)

In [ ]:
music_df = music_df_bk
if (debug >= 2):
    print('Parsing stem into band and post-band...')
music_df.loc[music_df['stem'].apply(lambda x: re.search('\d', x)).notnull(), 'band_abbrev_autoparsed'] = music_df['stem'][music_df['stem'].apply(lambda x: re.search('\d', x)).notnull()].apply(lambda x: x[0:re.search('\d',x).start()])
music_df['postbandstuff'] = music_df['stem'][music_df['stem'].apply(lambda x: re.search('\d+', x)).notnull()].apply(lambda x: x[re.search('\d', x).start():len(x)])

# If no digits found, everything is part of the band_abbrev_autoparsed
if (debug >= 2):
    print('Fixing band values for stems with no digits...')

music_df['band_abbrev_autoparsed']  = music_df['band_abbrev_autoparsed'].fillna(music_df['stem'])
music_df['postbandstuff']  = music_df['postbandstuff'].fillna('')

# Fix the band U2
if (debug >= 2):
    print('Fixing band values for U2...')

music_df.loc[
    music_df['band_abbrev_autoparsed'].apply(lambda x: x.lower() == 'u')
    , 'band_abbrev_autoparsed'
] = music_df['stem'][
    music_df['band_abbrev_autoparsed'].apply(lambda x: x.lower() == 'u')
].apply(lambda x: x[0:re.search('2[\s_-]*', x).end()])

music_df.loc[
    music_df['band_abbrev_autoparsed'].apply(lambda x: re.search('u2', x.lower())).notnull()
    , 'postbandstuff'
] = music_df['stem'][
    music_df['band_abbrev_autoparsed'].apply(lambda x: re.search('u2', x.lower())).notnull()
].apply(lambda x: x[re.search('2[\s_-]*', x).end():len(x)])

music_df_bk = music_df
music_df.sample(3)

In [ ]:
#music_df = music_df_bk
#                    ].dropna().drop_duplicates().sort_values()[0:30]#.values.tolist()

# Check for spacer characters at end of autoparsed band name
bandendspacers = re.compile('[\W_]+$')
music_df['hasbandendspacers'] = True
music_df.loc[music_df['band_abbrev_autoparsed'].apply(lambda x: re.search(bandendspacers, x)).isnull(), 'hasbandendspacers'] = False

music_df['band_abbrev'] = music_df['band_abbrev_autoparsed']

# Fix Loreley
music_df.loc[music_df['band_abbrev'].apply(lambda x: 'loreley' in x.lower()), 'band_abbrev'
            ] = music_df['band_abbrev'][music_df['band_abbrev'
                                                ].apply(lambda x: 'loreley' in x.lower())
                                       ].apply(lambda x: x[0:len(x)-8])

# Remove trailing spaces from band_abbrev_autoparsed to create band_abbrev
music_df.loc[
    (music_df['hasbandendspacers']) 
    , 'band_abbrev'] = music_df['band_abbrev'][
    (music_df['hasbandendspacers']) 
].apply(lambda x: x[0:re.search(bandendspacers, x).start()])


music_df['band_abbrev'] = music_df['band_abbrev'].apply(lambda x: x.lower().strip())

#music_df_bk = music_df
#music_df.loc['611156']
#music_df.sample(10)

# Start classifying bands by auto-finding not bands
#music_df = music_df_bk

starts_song_like = re.compile('^(track|dis(c|k) |side|tit(le|el))+')
says_song = re.compile('(?i)[\s\-\._]*(br\-|dis(c|k) |tit[le|el])[\s\-\._]*')
says_track = re.compile('(?i)^[-|cd]*[\s]*(track|side)+')
#says_track = re.compile('(track)+')
unknown_like = re.compile('(?i)\d*[\s\.\-_\(\[]*(unknown|artist|title)')
intro_like = re.compile('(intro|audience|stage|banter)+')
weird_like = re.compile('(\>|\[|!|:|\*)+')
starts_with_dash = re.compile('^\s*-+\s*')

cdthing = re.compile('(cd)+')

music_df['band'] = ''
music_df['month_in_band_abbrev'] = ''

music_df.loc[music_df['band_abbrev'] == '', 'band'] = 'blank'
music_df.loc[music_df['band_abbrev'].apply(lambda x: re.search(starts_song_like, x)).notnull(), 'band'] = 'notband'#'starts_song_like'
music_df.loc[music_df['band_abbrev'].apply(lambda x: re.search(says_song, x)).notnull(), 'band'] = 'notband'#'says_song'
music_df.loc[music_df['band_abbrev'].apply(lambda x: re.search(says_track, x)).notnull(), 'band'] = 'notband'#'says_track'
music_df.loc[music_df['band_abbrev'].apply(lambda x: re.search(unknown_like, x)).notnull(), 'band'] = 'notband'#'unknown_like'
music_df.loc[music_df['band_abbrev'].apply(lambda x: re.search(intro_like, x)).notnull(), 'band'] = 'notband'#'intro_like'
music_df.loc[music_df['band_abbrev'].apply(lambda x: re.search(weird_like, x)).notnull(), 'band'] = 'notband'#'weird_like'
music_df.loc[music_df['band_abbrev'].apply(lambda x: re.search(starts_with_dash, x)).notnull(), 'band'] = 'notband'#'starts_with_dash'
music_df.loc[music_df['band_abbrev'] == 'cd', 'band'] = 'notband'#'justsayscd'

music_df = fixbands(music_df)

music_df_bk = music_df

print('ok')

#music_df['band_abbrev'][(music_df['band'] == '') & (music_df['band_abbrev']).apply(lambda x: re.search(cdthing, x)).notnull()].drop_duplicates().sort_values()

In [ ]:
def fixbands(df):

    df.loc[
        df['band_abbrev'] == '_slash_high voltage', 
        ['band_abbrev', 'hasbandendspacers']
    ] = ['_slash', True]
    
    df.loc[df['band_abbrev'] == 'ac-dc_nov', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['ac-dc', True, 'nov']
    df.loc[df['band_abbrev'] == 'acquired as stand-alone flac stereolab-colchester arts centre', ['band_abbrev', 'hasbandendspacers']] = ['stereolab', True]
    df.loc[df['band_abbrev'] == 'acquired as stand-alone flac stereolab-manchester university', ['band_abbrev', 'hasbandendspacers']] = ['stereolab', True]
    df.loc[df['band_abbrev'] == 'acdcamsterdam', ['band_abbrev', 'hasbandendspacers']] = ['acdc', True]
    df.loc[df['band_abbrev'] == 'adele,st jordi', ['band_abbrev', 'hasbandendspacers']] = ['adele', True]
    df.loc[df['band_abbrev'] == 'aerosmith oracleworld', ['band_abbrev', 'hasbandendspacers']] = ['aerosmith', True]
    df.loc[df['band_abbrev'] == 'afghan whigs - leeds', ['band_abbrev', 'hasbandendspacers']] = ['afghan whigs', True]
    df.loc[df['band_abbrev'] == 'afl & low riders royal albert hall london england june', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['afl & low riders', True, 'june']
    df.loc[df['band_abbrev'] == 'aimee mann christmas', ['band_abbrev', 'hasbandendspacers']] = ['aimee mann', True]
    df.loc[df['band_abbrev'] == 'aimee mann_christmas', ['band_abbrev', 'hasbandendspacers']] = ['aimee mann', True]
    df.loc[df['band_abbrev'] == 'alain johannes gamh sf ca', ['band_abbrev', 'hasbandendspacers']] = ['alain johannes', True]
    df.loc[df['band_abbrev'] == 'alice cooper - august', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['alice cooper', True, 'august']
    df.loc[df['band_abbrev'] == 'alicecooperffm', ['band_abbrev', 'hasbandendspacers']] = ['alicecooper', True]
    df.loc[df['band_abbrev'] == 'alicecooperludwigsburg', ['band_abbrev', 'hasbandendspacers']] = ['alicecooper', True]
    df.loc[df['band_abbrev'] == 'alisonkraussunionstation', ['band_abbrev', 'hasbandendspacers']] = ['alisonkrauss', True]
    df.loc[df['band_abbrev'] == 'anapopovicbensheim', ['band_abbrev', 'hasbandendspacers']] = ['anapopovic', True]
    df.loc[df['band_abbrev'] == 'andy irvine - hamburg', ['band_abbrev', 'hasbandendspacers']] = ['andy irvine', True]
    df.loc[df['band_abbrev'] == 'ani difranco zellerbach berkeley ca', ['band_abbrev', 'hasbandendspacers']] = ['ani difranco', True]
    df.loc[df['band_abbrev'] == 'bbc con orch in queen friday night, at the opera', ['band_abbrev', 'hasbandendspacers']] = ['bbc con orch', True]
    df.loc[df['band_abbrev'] == 'bbking granada', ['band_abbrev', 'hasbandendspacers']] = ['bbking', True]
    df.loc[df['band_abbrev'] == 'bbking valencia', ['band_abbrev', 'hasbandendspacers']] = ['bbking', True]
    df.loc[df['band_abbrev'] == 'bbs-oct-man', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['bbs', True, 'oct']
    df.loc[df['band_abbrev'] == 'bd july', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['bd', True, 'july']
    df.loc[df['band_abbrev'] == 'bd-nov', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['bd', True, 'nov']
    df.loc[df['band_abbrev'] == 'beastie boys - manchester evening news', ['band_abbrev', 'hasbandendspacers']] = ['beastie boys', True]
    df.loc[df['band_abbrev'] == 'beck - august', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['beck', True, 'august']
    df.loc[df['band_abbrev'] == 'beethoven_paavo jaervi_sept', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['paavo jaervi', True, 'sept']
    df.loc[df['band_abbrev'] == 'behemoth phoenix', ['band_abbrev', 'hasbandendspacers']] = ['behemoth', True]
    df.loc[df['band_abbrev'] == 'benefit for anya phillips', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'best coast fox theater oakland ca may', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['best coast', True, 'may']
    df.loc[df['band_abbrev'] == 'big business - boulder co', ['band_abbrev', 'hasbandendspacers']] = ['big business', True]
    df.loc[df['band_abbrev'] == 'billy joel - november', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['billy joel', True, 'november']
    df.loc[df['band_abbrev'] == 'black sabbath bayou city theater houston texas march', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['black sabbath', True, 'march']
    df.loc[df['band_abbrev'] == 'bob dylan  greek theater  berkeley', ['band_abbrev', 'hasbandendspacers']] = ['bob dylan', True]
    df.loc[df['band_abbrev'] == 'bob dylan - augusta', ['band_abbrev', 'hasbandendspacers']] = ['bob dylan', True]
    df.loc[df['band_abbrev'] == 'bob dylan - genoa', ['band_abbrev', 'hasbandendspacers']] = ['bob dylan', True]
    df.loc[df['band_abbrev'] == 'bob dylan - gulf coast highway_disc', ['band_abbrev', 'hasbandendspacers']] = ['bob dylan', True]
    df.loc[df['band_abbrev'] == 'bob dylan - lewiston august', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['bob dylan', True, 'august']
    df.loc[df['band_abbrev'] == 'bob dylan - october', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['bob dylan', True, 'october']
    df.loc[df['band_abbrev'] == 'bob dylan - track', ['band_abbrev', 'hasbandendspacers']] = ['bob dylan', True]
    df.loc[df['band_abbrev'] == 'bob dylan greek theater berkeley, ca', ['band_abbrev', 'hasbandendspacers']] = ['bob dylan', True]
    df.loc[df['band_abbrev'] == 'bob dylan shoreline oct', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['bob dylan', True, 'oct']    
    df.loc[df['band_abbrev'] == 'bob marley & the wailers', ['band_abbrev', 'hasbandendspacers']] = ['bob dylan', True]
    df.loc[df['band_abbrev'] == 'bob marley + wailers', ['band_abbrev', 'hasbandendspacers']] = ['bob dylan', True]
    df.loc[df['band_abbrev'] == 'bob marley - is this love - mono -mixdown', ['band_abbrev', 'hasbandendspacers']] = ['bob marley', True]
    df.loc[df['band_abbrev'] == 'bob marley - lively up yourself - mono mixdown', ['band_abbrev', 'hasbandendspacers']] = ['bob marley', True]
    df.loc[df['band_abbrev'] == 'bob marley - no woman no cry - mono mixdown', ['band_abbrev', 'hasbandendspacers']] = ['bob marley', True]
    df.loc[df['band_abbrev'] == 'bobdylan-jonesbeach', ['band_abbrev', 'hasbandendspacers']] = ['bobdylan', True]
    df.loc[df['band_abbrev'] == 'bonamassa sellersville', ['band_abbrev', 'hasbandendspacers']] = ['bonamassa', True]
    df.loc[df['band_abbrev'] == 'bongripper - denver co', ['band_abbrev', 'hasbandendspacers']] = ['bongripper', True]
    df.loc[df['band_abbrev'] == 'bonnie raitt  civic auditorium  santa cruz', ['band_abbrev', 'hasbandendspacers']] = ['bonnie raitt', True]
    df.loc[df['band_abbrev'] == 'bonnie raitt luther burbank auditorium', ['band_abbrev', 'hasbandendspacers']] = ['bonnie raitt', True]
    df.loc[df['band_abbrev'] == 'bonus track', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'british sea power - dunfermline', ['band_abbrev', 'hasbandendspacers']] = ['british sea power', True]
    df.loc[df['band_abbrev'] == 'british sea power-focus wales wrexham central station', ['band_abbrev', 'hasbandendspacers']] = ['british sea power', True]
    df.loc[df['band_abbrev'] == 'brixandtheextricated-glasgow', ['band_abbrev', 'hasbandendspacers']] = ['brixandtheextricated', True]
    df.loc[df['band_abbrev'] == 'broken_windows_d', ['band_abbrev', 'hasbandendspacers']] = ['broken_windows', True]
    df.loc[df['band_abbrev'] == 'bromberg ultrasonic', ['band_abbrev', 'hasbandendspacers']] = ['bromberg', True]
    df.loc[df['band_abbrev'] == 'bruce philly night two tk', ['band_abbrev', 'hasbandendspacers']] = ['bruce', True]
    df.loc[df['band_abbrev'] == 'bruce springsteen- antwerpen', ['band_abbrev', 'hasbandendspacers']] = ['bruce springsteen', True]
    df.loc[df['band_abbrev'] == 'bt-disc', ['band_abbrev', 'hasbandendspacers']] = ['bt', True]
    df.loc[df['band_abbrev'] == 'buddy guy marin civic center nov', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['buddy guy', True, 'nov']    
    df.loc[df['band_abbrev'] == 'buddy guy wuth eric clapton the limelight', ['band_abbrev', 'hasbandendspacers']] = ['buddy guy wuth eric clapton', True]
    df.loc[df['band_abbrev'] == 'budgie bierkeller bristol', ['band_abbrev', 'hasbandendspacers']] = ['budgie bierkeller', True]
    df.loc[df['band_abbrev'] == 'bush tetras', ['band_abbrev', 'hasbandendspacers']] = ['bush', True]
    df.loc[df['band_abbrev'] == 'bvcombo_xmas', ['band_abbrev', 'hasbandendspacers']] = ['bvcombo', True]
    df.loc[df['band_abbrev'] == 'byrds bloomington', ['band_abbrev', 'hasbandendspacers']] = ['byrds', True]
    df.loc[df['band_abbrev'] == 'caravan - record plant', ['band_abbrev', 'hasbandendspacers']] = ['caravan', True]
    df.loc[df['band_abbrev'] == 'cardiacs - stonehenge', ['band_abbrev', 'hasbandendspacers']] = ['cardiacs', True]
    df.loc[df['band_abbrev'] == 'catfish & the bottlemen - edinburgh', ['band_abbrev', 'hasbandendspacers']] = ['catfish & the bottlemen', True]
    df.loc[df['band_abbrev'] == 'ccr berlin', ['band_abbrev', 'hasbandendspacers']] = ['ccr', True]
    df.loc[df['band_abbrev'] == 'cdg best buy', ['band_abbrev', 'hasbandendspacers']] = ['cdg', True]
    df.loc[df['band_abbrev'] == 'cdg chicago sbd', ['band_abbrev', 'hasbandendspacers']] = ['cdg', True]
    df.loc[df['band_abbrev'] == 'chameleons vox-clitheroe grand', ['band_abbrev', 'hasbandendspacers']] = ['chameleons', True]
    df.loc[df['band_abbrev'] == 'chameleons vox-manchester ritz', ['band_abbrev', 'hasbandendspacers']] = ['chameleons', True]
    df.loc[df['band_abbrev'] == 'chameleons vox-sheffield plug', ['band_abbrev', 'hasbandendspacers']] = ['chameleons', True]
    df.loc[df['band_abbrev'] == 'cheap trick phoenix', ['band_abbrev', 'hasbandendspacers']] = ['cheap trick', True]
    df.loc[df['band_abbrev'] == 'cherry glazerr commodore ballroom vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['cherry glazerr', True]
    df.loc[df['band_abbrev'] == 'chicago transit authority - fillmore west d', ['band_abbrev', 'hasbandendspacers']] = ['chicago transit authority', True]
    df.loc[df['band_abbrev'] == 'chris whitley - live seattle-fm', ['band_abbrev', 'hasbandendspacers']] = ['chris whitley', True]
    df.loc[df['band_abbrev'] == 'city of lights - stevie ray vaughan', ['band_abbrev', 'hasbandendspacers']] = ['stevie ray vaughan', True]
    df.loc[df['band_abbrev'] == 'clapton_double_layla', ['band_abbrev', 'hasbandendspacers']] = ['clapton', True]
    df.loc[df['band_abbrev'] == 'clapton_final_chapter', ['band_abbrev', 'hasbandendspacers']] = ['clapton', True]
    df.loc[df['band_abbrev'] == 'clarence_white_t', ['band_abbrev', 'hasbandendspacers']] = ['clarence_white', True]
    df.loc[df['band_abbrev'] == 'cobain_old-age', ['band_abbrev', 'hasbandendspacers']] = ['cobain', True]
    df.loc[df['band_abbrev'] == 'creeps fr -jazzhaus  may', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['creeps', True, 'may']
    df.loc[df['band_abbrev'] == 'crowded house womad sf ca sep', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['crowded house', True, 'sep']
    df.loc[df['band_abbrev'] == 'cspedding lsc', ['band_abbrev', 'hasbandendspacers']] = ['cspedding', True]
    df.loc[df['band_abbrev'] == 'cult of luna with julie christmas - las vegas nv', ['band_abbrev', 'hasbandendspacers']] = ['cult of luna with julie christmas', True]
    df.loc[df['band_abbrev'] == 'd', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'd perez - j patitucci - t l carrington talk', ['band_abbrev', 'hasbandendspacers']] = ['d perez - j patitucci - t l carrington', True]
    df.loc[df['band_abbrev'] == 'd+t', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'damianwilsonreichenbach', ['band_abbrev', 'hasbandendspacers']] = ['damianwilson', True]
    df.loc[df['band_abbrev'] == 'darlene love   stern grove festival', ['band_abbrev', 'hasbandendspacers']] = ['darlene love', True]
    df.loc[df['band_abbrev'] == 'daryl hall & john oates - het vondelpark, amsterdam', ['band_abbrev', 'hasbandendspacers']] = ['daryl hall & john oates', True]
    df.loc[df['band_abbrev'] == 'dave holland trio; november', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['dave holland trio', True, 'november']
    df.loc[df['band_abbrev'] == 'david grisman birthday bash', ['band_abbrev', 'hasbandendspacers']] = ['david grisman', True]
    df.loc[df['band_abbrev'] == 'david lindley dunsmuir house oakand ca aug', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['david lindley', True, 'aug']
    df.loc[df['band_abbrev'] == 'dead cross_denver co', ['band_abbrev', 'hasbandendspacers']] = ['dead cross', True]
    df.loc[df['band_abbrev'] == 'death valley girls astoria hotel vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['death valley girls', True]
    df.loc[df['band_abbrev'] == 'deep purple - august', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['deep purple', True, 'august']
    df.loc[df['band_abbrev'] == 'deep purple - no', ['band_abbrev', 'hasbandendspacers']] = ['deep purple', True]
    df.loc[df['band_abbrev'] == 'delivery - bbc sunday concert', ['band_abbrev', 'hasbandendspacers']] = ['delivery', True]
    df.loc[df['band_abbrev'] == 'delivery - brighton', ['band_abbrev', 'hasbandendspacers']] = ['delivery', True]
    df.loc[df['band_abbrev'] == 'delivery - music in the moat', ['band_abbrev', 'hasbandendspacers']] = ['delivery', True]
    df.loc[df['band_abbrev'] == 'der nino aus wien', ['band_abbrev', 'hasbandendspacers']] = ['der nino', True]
    df.loc[df['band_abbrev'] == 'dermot kennedy - manchester', ['band_abbrev', 'hasbandendspacers']] = ['dermot kennedy', True]
    df.loc[df['band_abbrev'] == 'different_degrees_d', ['band_abbrev', 'hasbandendspacers']] = ['different_degrees', True]
    df.loc[df['band_abbrev'] == 'dinosaur jr_denver co', ['band_abbrev', 'hasbandendspacers']] = ['dinosaur jr', True]
    df.loc[df['band_abbrev'] == 'dkcols.flacs/kcolst', ['band_abbrev', 'hasbandendspacers']] = ['dkcols', True]
    df.loc[df['band_abbrev'] == 'donovan  regency ballroom  sf', ['band_abbrev', 'hasbandendspacers']] = ['donovan', True]
    df.loc[df['band_abbrev'] == 'doobie bros-botanic park', ['band_abbrev', 'hasbandendspacers']] = ['doobie bros', True]
    df.loc[df['band_abbrev'] == 'doobie brothers shoreline amphitheater jun', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['doobie brothers', True, 'jun']
    df.loc[df['band_abbrev'] == 'doromuenchen', ['band_abbrev', 'hasbandendspacers']] = ['doro', True]
    df.loc[df['band_abbrev'] == 'dream syndicate  the independent', ['band_abbrev', 'hasbandendspacers']] = ['dream syndicate', True]
    df.loc[df['band_abbrev'] == 'dri - october', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['dri', True, 'october']
    df.loc[df['band_abbrev'] == 'drjohn - june', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['drjohn', True, 'june']
    df.loc[df['band_abbrev'] == 'dtb-disc', ['band_abbrev', 'hasbandendspacers']] = ['dtb', True]
    df.loc[df['band_abbrev'] == 'dylan - exclusive series', ['band_abbrev', 'hasbandendspacers']] = ['dylan', True]
    df.loc[df['band_abbrev'] == 'dylan - restless hungry feelings_disc', ['band_abbrev', 'hasbandendspacers']] = ['dylan', True]    
    df.loc[df['band_abbrev'] == 'dylan essen', ['band_abbrev', 'hasbandendspacers']] = ['dylan', True]
    df.loc[df['band_abbrev'] == 'dylan hh', ['band_abbrev', 'hasbandendspacers']] = ['dylan', True]
    df.loc[df['band_abbrev'] == 'dylan-mai', ['band_abbrev', 'hasbandendspacers']] = ['dylan', True]
    df.loc[df['band_abbrev'] == 'dylan_aalborg', ['band_abbrev', 'hasbandendspacers']] = ['dylan', True]
    df.loc[df['band_abbrev'] == 'dylanspokane', ['band_abbrev', 'hasbandendspacers']] = ['dylan', True]
    df.loc[df['band_abbrev'] == 'e', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'eb_nac', ['band_abbrev', 'hasbandendspacers']] = ['eb', True]
    df.loc[df['band_abbrev'] == 'ec - american tour', ['band_abbrev', 'hasbandendspacers']] = ['ec', True]
    df.loc[df['band_abbrev'] == 'ec - hideaway', ['band_abbrev', 'hasbandendspacers']] = ['ec', True]
    df.loc[df['band_abbrev'] == 'ec-msg', ['band_abbrev', 'hasbandendspacers']] = ['ec', True]
    df.loc[df['band_abbrev'] == 'ecsw izod arena east rutherford nj', ['band_abbrev', 'hasbandendspacers']] = ['ecsw', True]
    df.loc[df['band_abbrev'] == 'ecsw lg arena birmingham may', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['ecsw', True, 'may']
    df.loc[df['band_abbrev'] == 'ecsw mgm grand arena las vegas', ['band_abbrev', 'hasbandendspacers']] = ['ecsw', True]
    df.loc[df['band_abbrev'] == 'ecsw oracle arena oakland ca', ['band_abbrev', 'hasbandendspacers']] = ['ecsw', True]
    df.loc[df['band_abbrev'] == 'ecsw wachovia center', ['band_abbrev', 'hasbandendspacers']] = ['ecsw', True]
    df.loc[df['band_abbrev'] == 'ecsw wembley arena london england may', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['ecsw', True, 'may']
    df.loc[df['band_abbrev'] == 'ed&jh-wbcn-boston', ['band_abbrev', 'hasbandendspacers']] = ['ed&jh', True]
    df.loc[df['band_abbrev'] == 'edgar winter band - august', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['edgar winter band', True, 'aug']
    df.loc[df['band_abbrev'] == 'edie brickel & the new bohemians  zellerbach aud berkeley ca apr', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['edie brickel & the new bohemians', True, 'apr']
    df.loc[df['band_abbrev'] == 'elder - edinburgh', ['band_abbrev', 'hasbandendspacers']] = ['elder', True]
    df.loc[df['band_abbrev'] == 'elton john - rocket man d', ['band_abbrev', 'hasbandendspacers']] = ['elton john', True]
    df.loc[df['band_abbrev'] == 'elton john_earls court', ['band_abbrev', 'hasbandendspacers']] = ['elton john', True]
    df.loc[df['band_abbrev'] == 'em antequera', ['band_abbrev', 'hasbandendspacers']] = ['em', True]
    df.loc[df['band_abbrev'] == 'em estepona', ['band_abbrev', 'hasbandendspacers']] = ['em', True]
    df.loc[df['band_abbrev'] == 'em malaga', ['band_abbrev', 'hasbandendspacers']] = ['em', True]
    df.loc[df['band_abbrev'] == 'em proms', ['band_abbrev', 'hasbandendspacers']] = ['em', True]
    df.loc[df['band_abbrev'] == 'ema fox cabaret vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['ema fox', True]
    df.loc[df['band_abbrev'] == 'emilly remler boston, ma', ['band_abbrev', 'hasbandendspacers']] = ['emily remler', True]
    df.loc[df['band_abbrev'] == 'emily haines and the soft skeleton chan centre  vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['emily haines and the soft skeleton', True]
    df.loc[df['band_abbrev'] == 'ernesto bladden iv kpri', ['band_abbrev', 'hasbandendspacers']] = ['ernesto bladden', True]
    df.loc[df['band_abbrev'] == 'evanescence midland theatre kansas city missouri april', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['evanescence', True, 'april']
    df.loc[df['band_abbrev'] == 'evidence_d', ['band_abbrev', 'hasbandendspacers']] = ['evidence', True]
    df.loc[df['band_abbrev'] == 'eyehategod_denver co', ['band_abbrev', 'hasbandendspacers']] = ['eyehategod', True]
    df.loc[df['band_abbrev'] == 'fall-roskilde', ['band_abbrev', 'hasbandendspacers']] = ['fall', True]
    df.loc[df['band_abbrev'] == 'fixedtrack', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'flac', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'flac without noise reduction', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'foo fighters - washington dc', ['band_abbrev', 'hasbandendspacers']] = ['foo fighters', True]
    df.loc[df['band_abbrev'] == 'g-grassgospelhour', ['band_abbrev', 'hasbandendspacers']] = ['g-grass', True]
    df.loc[df['band_abbrev'] == 'gaf-disc', ['band_abbrev', 'hasbandendspacers']] = ['gaf', True]
    df.loc[df['band_abbrev'] == 'galay bustos ccsm  ba', ['band_abbrev', 'hasbandendspacers']] = ['galay bustos', True]
    df.loc[df['band_abbrev'] == 'gd-disc', ['band_abbrev', 'hasbandendspacers']] = ['gd', True]
    df.loc[df['band_abbrev'] == 'gd-pigpenmixd', ['band_abbrev', 'hasbandendspacers']] = ['gd', True]
    df.loc[df['band_abbrev'] == 'gfr-disc', ['band_abbrev', 'hasbandendspacers']] = ['gfr', True]
    df.loc[df['band_abbrev'] == 'gh_far east man_t', ['band_abbrev', 'hasbandendspacers']] = ['gh_far east', True]
    df.loc[df['band_abbrev'] == 'gift-to-tapers', ['band_abbrev', 'hasbandendspacers']] = ['gift', True]
    df.loc[df['band_abbrev'] == 'glen hansard - live', ['band_abbrev', 'hasbandendspacers']] = ['glen hansard', True]
    df.loc[df['band_abbrev'] == 'glen hansard - prague', ['band_abbrev', 'hasbandendspacers']] = ['glen hansard', True]
    df.loc[df['band_abbrev'] == 'glenn hughes - burning warsaw night', ['band_abbrev', 'hasbandendspacers']] = ['glenn hughes', True]
    df.loc[df['band_abbrev'] == 'gm-disc', ['band_abbrev', 'hasbandendspacers']] = ['gm', True]
    df.loc[df['band_abbrev'] == 'gnr-nr-t', ['band_abbrev', 'hasbandendspacers']] = ['gnr', True]
    df.loc[df['band_abbrev'] == 'gov\'tmule.humphreys.sdca', ['band_abbrev', 'hasbandendspacers']] = ['gov\t mule', True]
    df.loc[df['band_abbrev'] == 'gov\'tmule.ventura theater', ['band_abbrev', 'hasbandendspacers']] = ['gov\t mule', True]
    df.loc[df['band_abbrev'] == 'graceful meadowlands d', ['band_abbrev', 'hasbandendspacers']] = ['graceful', True]
    df.loc[df['band_abbrev'] == 'guillermoklein-dc', ['band_abbrev', 'hasbandendspacers']] = ['guillermoklein', True]
    df.loc[df['band_abbrev'] == 'gurge-latwanadalesydney', ['band_abbrev', 'hasbandendspacers']] = ['gurge', True]
    df.loc[df['band_abbrev'] == 'haas in vain @ vcolon  ba', ['band_abbrev', 'hasbandendspacers']] = ['haas in vain', True]
    df.loc[df['band_abbrev'] == 'hanging stars track', ['band_abbrev', 'hasbandendspacers']] = ['hanging stars', True]
    df.loc[df['band_abbrev'] == 'hawkwind new theater oxford uk november', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['hawkwind', True, True]
    df.loc[df['band_abbrev'] == 'hawkwind tower theater philadelphia pennsylvania april', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['hawkwind', True, True]
    df.loc[df['band_abbrev'] == 'helmetofgnatsprogday', ['band_abbrev', 'hasbandendspacers']] = ['helmet of gnats', True]
    df.loc[df['band_abbrev'] == 'hol preston live', ['band_abbrev', 'hasbandendspacers']] = ['hol', True]
    df.loc[df['band_abbrev'] == 'hubbard_boston', ['band_abbrev', 'hasbandendspacers']] = ['hubbard', True]
    df.loc[df['band_abbrev'] == 'hugh masekela & larry willis, rfh', ['band_abbrev', 'hasbandendspacers']] = ['hugh masekela & larry willis', True]
    df.loc[df['band_abbrev'] == 'hugh masekela winterthur', ['band_abbrev', 'hasbandendspacers']] = ['hugh masekela', True]
    df.loc[df['band_abbrev'] == 'human league-liverpool clarence dock', ['band_abbrev', 'hasbandendspacers']] = ['human league', True]
    df.loc[df['band_abbrev'] == 'icp toronto', ['band_abbrev', 'hasbandendspacers']] = ['icp', True]
    df.loc[df['band_abbrev'] == 'icporchestra-amsterdam', ['band_abbrev', 'hasbandendspacers']] = ['icporchestra', True]
    df.loc[df['band_abbrev'] == 'icporchestra-geneve', ['band_abbrev', 'hasbandendspacers']] = ['icporchestra', True]
    df.loc[df['band_abbrev'] == 'idlewild - glasgow', ['band_abbrev', 'hasbandendspacers']] = ['idlewild', True]
    df.loc[df['band_abbrev'] == 'ig columbus', ['band_abbrev', 'hasbandendspacers']] = ['ig', True]
    df.loc[df['band_abbrev'] == 'instant_attraction_d', ['band_abbrev', 'hasbandendspacers']] = ['instant_attraction', True]
    df.loc[df['band_abbrev'] == 'interpol_lille', ['band_abbrev', 'hasbandendspacers']] = ['interpol', True]
    df.loc[df['band_abbrev'] == 'interview band members', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'inxsalbert', ['band_abbrev', 'hasbandendspacers']] = ['inxs', True]
    df.loc[df['band_abbrev'] == 'iron & wine    warfield theater', ['band_abbrev', 'hasbandendspacers']] = ['iron & wine', True]
    df.loc[df['band_abbrev'] == 'ja-remaster', ['band_abbrev', 'hasbandendspacers']] = ['ja', True]
    df.loc[df['band_abbrev'] == 'ja_set', ['band_abbrev', 'hasbandendspacers']] = ['ja', True]
    df.loc[df['band_abbrev'] == 'jaap blonk solo @unacasa ba', ['band_abbrev', 'hasbandendspacers']] = ['jaap blonk', True]
    df.loc[df['band_abbrev'] == 'jack&ginger-cleveland-tr', ['band_abbrev', 'hasbandendspacers']] = ['jack&ginger', True]
    df.loc[df['band_abbrev'] == 'jandek - berlin germany', ['band_abbrev', 'hasbandendspacers']] = ['jandek', True]
    df.loc[df['band_abbrev'] == 'jared james nichols - october', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['jared james nichols', True, 'october']
    df.loc[df['band_abbrev'] == 'jason bonham lze indianapolis', ['band_abbrev', 'hasbandendspacers']] = ['jason bonham', True]
    df.loc[df['band_abbrev'] == 'jeff beck  paramount', ['band_abbrev', 'hasbandendspacers']] = ['jeff beck', True]
    df.loc[df['band_abbrev'] == 'jerry jeff walker & david bromberg wbai', ['band_abbrev', 'hasbandendspacers']] = ['jerry jeff walker & david bromberg', True]
    df.loc[df['band_abbrev'] == 'jherring albany', ['band_abbrev', 'hasbandendspacers']] = ['jherring', True]
    df.loc[df['band_abbrev'] == 'jimmy herring-cranston', ['band_abbrev', 'hasbandendspacers']] = ['jimmy herring', True]
    df.loc[df['band_abbrev'] == 'john mclaughlin and jimmy herring - november', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['john mclaughlin and jimmy herring', True, 'november']
    df.loc[df['band_abbrev'] == 'john moreland warfield theater', ['band_abbrev', 'hasbandendspacers']] = ['john moreland', True]
    df.loc[df['band_abbrev'] == 'john prine palladium cd', ['band_abbrev', 'hasbandendspacers']] = ['john prine', True]
    df.loc[df['band_abbrev'] == 'johnny thunders so alone revue', ['band_abbrev', 'hasbandendspacers']] = ['johnny thunder', True]
    df.loc[df['band_abbrev'] == 'jt-disc', ['band_abbrev', 'hasbandendspacers']] = ['jt', True]
    df.loc[df['band_abbrev'] == 'jw-estadio revolucion', ['band_abbrev', 'hasbandendspacers']] = ['jw', True]
    df.loc[df['band_abbrev'] == 'karine polwart - vienna, austria', ['band_abbrev', 'hasbandendspacers']] = ['karine polwart', True]
    df.loc[df['band_abbrev'] == 'keith richards sala zeleste', ['band_abbrev', 'hasbandendspacers']] = ['keith richards', True]
    df.loc[df['band_abbrev'] == 'killer country  love song bar   la', ['band_abbrev', 'hasbandendspacers']] = ['killer country', True]
    df.loc[df['band_abbrev'] == 'kim fowley - primitive', ['band_abbrev', 'hasbandendspacers']] = ['kim fowley', True]
    df.loc[df['band_abbrev'] == 'king diamonds - live chile', ['band_abbrev', 'hasbandendspacers']] = ['king diamonds - live chile', True]
    df.loc[df['band_abbrev'] == 'kinksphilhalny', ['band_abbrev', 'hasbandendspacers']] = ['kinks', True]
    df.loc[df['band_abbrev'] == 'kool-aid s', ['band_abbrev', 'hasbandendspacers']] = ['kool-aid', True]
    df.loc[df['band_abbrev'] == 'kp set', ['band_abbrev', 'hasbandendspacers']] = ['kp', True]
    df.loc[df['band_abbrev'] == 'kravitz - lenny\'s birthday', ['band_abbrev', 'hasbandendspacers']] = ['kravitz', True]
    df.loc[df['band_abbrev'] == 'lamb of god phoenix', ['band_abbrev', 'hasbandendspacers']] = ['lamb of god', True]
    df.loc[df['band_abbrev'] == 'lcd soundsystem - washington dc', ['band_abbrev', 'hasbandendspacers']] = ['lcd soundsystem', True]
    df.loc[df['band_abbrev'] == 'le_gran_detuning_d', ['band_abbrev', 'hasbandendspacers']] = ['le_gran_detuning', True]
    df.loc[df['band_abbrev'] == 'leaveseyesstuttgart', ['band_abbrev', 'hasbandendspacers']] = ['leaveseyes', True]
    df.loc[df['band_abbrev'] == 'lefly-kassel', ['band_abbrev', 'hasbandendspacers']] = ['lefly', True]
    df.loc[df['band_abbrev'] == 'les filles de illighadad luzern', ['band_abbrev', 'hasbandendspacers']] = ['les filles de illighadad', True]
    df.loc[df['band_abbrev'] == 'leslie stevens  echoplex', ['band_abbrev', 'hasbandendspacers']] = ['leslie stevens', True]
    df.loc[df['band_abbrev'] == 'link wray - record plant', ['band_abbrev', 'hasbandendspacers']] = ['link wray', True]
    df.loc[df['band_abbrev'] == 'london contemp orch soloists', ['band_abbrev', 'hasbandendspacers']] = ['london contemp orch', True]
    df.loc[df['band_abbrev'] == 'love & rockets_november', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['love & rockets_november', True, 'november']
    df.loc[df['band_abbrev'] == 'lucero_denver co', ['band_abbrev', 'hasbandendspacers']] = ['lucero', True]
    df.loc[df['band_abbrev'] == 'lucia cadotsch speak low', ['band_abbrev', 'hasbandendspacers']] = ['lucia cadotsch', True]
    df.loc[df['band_abbrev'] == 'lucinda williams madrid', ['band_abbrev', 'hasbandendspacers']] = ['lucinda williams', True]
    df.loc[df['band_abbrev'] == 'maceo parker lugano', ['band_abbrev', 'hasbandendspacers']] = ['maceo parker', True]
    df.loc[df['band_abbrev'] == 'macula dog - kaleidoscopic flux radio show', ['band_abbrev', 'hasbandendspacers']] = ['macula dog', True]
    df.loc[df['band_abbrev'] == 'mahavishnu orchestra. yale university', ['band_abbrev', 'hasbandendspacers']] = ['mahavishnu orchestra', True]
    df.loc[df['band_abbrev'] == 'maidenunitedbadhomburg', ['band_abbrev', 'hasbandendspacers']] = ['maidenunited', True]
    df.loc[df['band_abbrev'] == 'maidenunitedddorf', ['band_abbrev', 'hasbandendspacers']] = ['maidenunited', True]
    df.loc[df['band_abbrev'] == 'malinky - sevenoaks', ['band_abbrev', 'hasbandendspacers']] = ['malinky', True]
    df.loc[df['band_abbrev'] == 'mannycharltonbandma', ['band_abbrev', 'hasbandendspacers']] = ['mannycharltonband', True]
    df.loc[df['band_abbrev'] == 'marillion_live in aucamville, salle georges brassens', ['band_abbrev', 'hasbandendspacers']] = ['marillion', True]
    df.loc[df['band_abbrev'] == 'mark geary - vienna, austria', ['band_abbrev', 'hasbandendspacers']] = ['mark geary', True]
    df.loc[df['band_abbrev'] == 'mark lanegan band - edinburgh', ['band_abbrev', 'hasbandendspacers']] = ['mark lanegan band', True]
    df.loc[df['band_abbrev'] == 'martin barre band    mystic theater  petaluma', ['band_abbrev', 'hasbandendspacers']] = ['martin barre band', True]
    df.loc[df['band_abbrev'] == 'martyn joseph - newport pc', ['band_abbrev', 'hasbandendspacers']] = ['martyn joseph', True]
    df.loc[df['band_abbrev'] == 'marvin gaye - aint no mountain - mono mixdown', ['band_abbrev', 'hasbandendspacers']] = ['marvin gaye', True]
    df.loc[df['band_abbrev'] == 'marvin gaye - aint nothing like the real thing...', ['band_abbrev', 'hasbandendspacers']] = ['marvin gaye', True]
    df.loc[df['band_abbrev'] == 'marvin gaye - mercy mercy me - mono mixdown', ['band_abbrev', 'hasbandendspacers']] = ['marvin gaye', True]
    df.loc[df['band_abbrev'] == 'matthaimovitz-dc', ['band_abbrev', 'hasbandendspacers']] = ['matthaimovitz', True]
    df.loc[df['band_abbrev'] == 'mavis staples - october', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['mavis staples', True, 'october']
    df.loc[df['band_abbrev'] == 'max\'s kansas city mar', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['max', True, 'mar']
    df.loc[df['band_abbrev'] == 'mellencamp frankfurt', ['band_abbrev', 'hasbandendspacers']] = ['mellencamp', True]
    df.loc[df['band_abbrev'] == 'michel godard_ track', ['band_abbrev', 'hasbandendspacers']] = ['michel godard', True]
    df.loc[df['band_abbrev'] == 'midnight oil   greek theater  la', ['band_abbrev', 'hasbandendspacers']] = ['midnight oil', True]
    df.loc[df['band_abbrev'] == 'ministry_riotfest', ['band_abbrev', 'hasbandendspacers']] = ['ministry', True]
    df.loc[df['band_abbrev'] == 'mix tape vol', ['band', 'hasbandendspacers']] = ['notfound', True]
    df.loc[df['band_abbrev'] == 'mms.cd', ['band_abbrev', 'hasbandendspacers']] = ['mms', True]
    df.loc[df['band_abbrev'] == 'mondosample sbd', ['band_abbrev', 'hasbandendspacers']] = ['mondosample', True]
    df.loc[df['band_abbrev'] == 'moody-blues-july', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['moody-blues', True, 'july']
    df.loc[df['band_abbrev'] == 'motley crue-track', ['band_abbrev', 'hasbandendspacers']] = ['motley crue', True]
    df.loc[df['band_abbrev'] == 'mschenker tor pontiac', ['band_abbrev', 'hasbandendspacers']] = ['mschenker tor', True]
    df.loc[df['band_abbrev'] == 'muddy waters cellar door wash dc july', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['muddy waters', True, 'july']
    df.loc[df['band_abbrev'] == 'nakedcity_warsaw', ['band_abbrev', 'hasbandendspacers']] = ['nakedcity', True]
    df.loc[df['band_abbrev'] == 'new mastersounds neptune theater', ['band_abbrev', 'hasbandendspacers']] = ['new mastersounds', True]
    df.loc[df['band_abbrev'] == 'new model army-holmfirth picturedrome', ['band_abbrev', 'hasbandendspacers']] = ['new model army', True]
    df.loc[df['band_abbrev'] == 'new order-transmission', ['band_abbrev', 'hasbandendspacers']] = ['new order', True]
    df.loc[df['band_abbrev'] == 'nickcave_copenhagen', ['band_abbrev', 'hasbandendspacers']] = ['nickcave', True]
    df.loc[df['band_abbrev'] == 'ny salsa all stars lugano', ['band_abbrev', 'hasbandendspacers']] = ['ny salsa all stars', True]
    df.loc[df['band_abbrev'] == 'omni_under_pressure_d', ['band_abbrev', 'hasbandendspacers']] = ['omni', True]
    df.loc[df['band_abbrev'] == 'operationmindcrimeaburg', ['band_abbrev', 'hasbandendspacers']] = ['operationmindcrime', True]
    df.loc[df['band_abbrev'] == 'oteil_friends pittsburgh', ['band_abbrev', 'hasbandendspacers']] = ['oteil_friends', True]
    df.loc[df['band_abbrev'] == 'ozrictentaclesdigitalpd', ['band_abbrev', 'hasbandendspacers']] = ['ozrictentacles', True]
    df.loc[df['band_abbrev'] == 'p,g msg d', ['band_abbrev', 'hasbandendspacers']] = ['p,g', True]
    df.loc[df['band_abbrev'] == 'pat  metheny. woodstock', ['band_abbrev', 'hasbandendspacers']] = ['pat  metheny', True]
    df.loc[df['band_abbrev'] == 'patrick street - salisbury', ['band_abbrev', 'hasbandendspacers']] = ['patrick street - salisbury', True]
    df.loc[df['band_abbrev'] == 'paul motian trio - village vanguard, nyc', ['band_abbrev', 'hasbandendspacers']] = ['paul motian trio', True]
    df.loc[df['band_abbrev'] == 'pearl jam (reel', ['band_abbrev', 'hasbandendspacers']] = ['pearl jam', True]
    df.loc[df['band_abbrev'] == 'permanent_tour_d', ['band_abbrev', 'hasbandendspacers']] = ['permanent_tour', True]
    df.loc[df['band_abbrev'] == 'petula clark  gallo center for the arts', ['band_abbrev', 'hasbandendspacers']] = ['petula clark', True]
    df.loc[df['band_abbrev'] == 'pf_birmingham', ['band_abbrev', 'hasbandendspacers']] = ['pf', True]
    df.loc[df['band_abbrev'] == 'ph d', ['band_abbrev', 'hasbandendspacers']] = ['ph', True]
    df.loc[df['band_abbrev'] == 'phil-disc', ['band_abbrev', 'hasbandendspacers']] = ['phil', True]
    df.loc[df['band_abbrev'] == 'pinetopperkins_piazzablues', ['band_abbrev', 'hasbandendspacers']] = ['pinetopperkins', True]
    df.loc[df['band_abbrev'] == 'pogues_hamburg', ['band_abbrev', 'hasbandendspacers']] = ['pogues', True]
    df.loc[df['band_abbrev'] == 'poptone_pabsttheater_sept', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['poptone', True, 'sept']
    df.loc[df['band_abbrev'] == 'por boston', ['band_abbrev', 'hasbandendspacers']] = ['por', True]
    df.loc[df['band_abbrev'] == 'potter, taborn, harland trio; december', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['potter, taborn, harland trio', True, 'december']
    df.loc[df['band_abbrev'] == 'psb norwich', ['band_abbrev', 'hasbandendspacers']] = ['psb', True]
    df.loc[df['band_abbrev'] == 'psychedelic furs - doncaster rotters (tb aud master', ['band_abbrev', 'hasbandendspacers']] = ['psychedelic furs - doncaster rotters', True]
    df.loc[df['band_abbrev'] == 'qotsa - october', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['qotsa', True, 'october']
    df.loc[df['band_abbrev'] == 'qotsa berlin norm', ['band_abbrev', 'hasbandendspacers']] = ['qotsa', True]
    df.loc[df['band_abbrev'] == 'qotsa manchester arena', ['band_abbrev', 'hasbandendspacers']] = ['qotsa', True]
    df.loc[df['band_abbrev'] == 'queen - bohemian rhapsody - mono mixdown', ['band_abbrev', 'hasbandendspacers']] = ['queen', True]
    df.loc[df['band_abbrev'] == 'queen - cologne', ['band_abbrev', 'hasbandendspacers']] = ['queen', True]
    df.loc[df['band_abbrev'] == 'quicksand_thaliahall_sept', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['quicksand', True, 'sept']
    df.loc[df['band_abbrev'] == 'r', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'r b', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'radiohead_lallapoloza_chicago_august', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['radiohead', True, 'august']
    df.loc[df['band_abbrev'] == 'ratt portland', ['band_abbrev', 'hasbandendspacers']] = ['ratt', True]
    df.loc[df['band_abbrev'] == 'reignwolf  cafe du nord   sf', ['band_abbrev', 'hasbandendspacers']] = ['reignwolf', True]
    df.loc[df['band_abbrev'] == 'rflack malaga', ['band_abbrev', 'hasbandendspacers']] = ['rflack', True]
    df.loc[df['band_abbrev'] == 'rg-unplug-d', ['band_abbrev', 'hasbandendspacers']] = ['rg', True]
    df.loc[df['band_abbrev'] == 'rhavens iow', ['band_abbrev', 'hasbandendspacers']] = ['rhavens', True]
    df.loc[df['band_abbrev'] == 'richard hawley - bury met', ['band_abbrev', 'hasbandendspacers']] = ['richard hawley', True]
    df.loc[df['band_abbrev'] == 'richard lloyd\'s summer jam at max\'s', ['band_abbrev', 'hasbandendspacers']] = ['richard lloyd', True]
    df.loc[df['band_abbrev'] == 'richard thompson  villa montalvo', ['band_abbrev', 'hasbandendspacers']] = ['richard thompson', True]
    df.loc[df['band_abbrev'] == 'ride - edinburgh', ['band_abbrev', 'hasbandendspacers']] = ['ride', True]
    df.loc[df['band_abbrev'] == 'rio berlin', ['band_abbrev', 'hasbandendspacers']] = ['rio', True]
    df.loc[df['band_abbrev'] == 'rio reiser kehl', ['band_abbrev', 'hasbandendspacers']] = ['rio', True]
    df.loc[df['band_abbrev'] == 'ripley music hall, philadelphia, pa', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'rkirk_tv_soul_t', ['band_abbrev', 'hasbandendspacers']] = ['rkirk', True]
    df.loc[df['band_abbrev'] == 'robert plant boston', ['band_abbrev', 'hasbandendspacers']] = ['robert plant', True]
    df.loc[df['band_abbrev'] == 'robert plant hammersmith apollo', ['band_abbrev', 'hasbandendspacers']] = ['robert plant', True]
    df.loc[df['band_abbrev'] == 'rodolphe burger + james blood ulmer - aã©ronef, lille, fr', ['band_abbrev', 'hasbandendspacers']] = ['rodolphe burger + james blood ulmer', True]
    df.loc[df['band_abbrev'] == 'rodolphe burger lomme', ['band_abbrev', 'hasbandendspacers']] = ['rodolphe burger', True]
    df.loc[df['band_abbrev'] == 'roger waters - boston', ['band_abbrev', 'hasbandendspacers']] = ['roger waters', True]
    df.loc[df['band_abbrev'] == 'rolling stones phoenix', ['band_abbrev', 'hasbandendspacers']] = ['rolling stones', True]
    df.loc[df['band_abbrev'] == 'roots-round', ['band_abbrev', 'hasbandendspacers']] = ['roots', True]
    df.loc[df['band_abbrev'] == 'rootsfriendsdallaspops', ['band_abbrev', 'hasbandendspacers']] = ['rootsfriends', True]
    df.loc[df['band_abbrev'] == 'roy harper jimmy page rock garden covent garden london england nov', ['band_abbrev', 'hasbandendspacers','month_in_band_abbrev']] = ['roy harper jimmy page', True, 'nov']
    df.loc[df['band_abbrev'] == 'roy harper jimmy page st ives hotel lytham st annes engand', ['band_abbrev', 'hasbandendspacers']] = ['roy harper jimmy page', True]
    df.loc[df['band_abbrev'] == 'royal blood - october', ['band_abbrev', 'hasbandendspacers','month_in_band_abbrev']] = ['royal blood', True, 'october']
    df.loc[df['band_abbrev'] == 'royal blood-manchester academy', ['band_abbrev', 'hasbandendspacers']] = ['royal blood', True]
    df.loc[df['band_abbrev'] == 'royal string quartet - tgsm baires', ['band_abbrev', 'hasbandendspacers']] = ['royal string quartet', True]
    df.loc[df['band_abbrev'] == 'rt albany', ['band_abbrev', 'hasbandendspacers']] = ['rt', True]
    df.loc[df['band_abbrev'] == 'rt band nyc', ['band_abbrev', 'hasbandendspacers']] = ['rt band', True]
    df.loc[df['band_abbrev'] == 'rt norwich', ['band_abbrev', 'hasbandendspacers']] = ['rt', True]
    df.loc[df['band_abbrev'] == 'rt santa cruz', ['band_abbrev', 'hasbandendspacers']] = ['rt', True]
    df.loc[df['band_abbrev'] == 'rt-disc', ['band_abbrev', 'hasbandendspacers']] = ['rt', True]
    df.loc[df['band_abbrev'] == 'runshort_d', ['band_abbrev', 'hasbandendspacers']] = ['runshort', True]
    df.loc[df['band_abbrev'] == 'rush - summer nights', ['band_abbrev', 'hasbandendspacers']] = ['rush', True]
    df.loc[df['band_abbrev'] == 'rush-july', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['rush', True, 'july']
    df.loc[df['band_abbrev'] == 'rush-march', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['rush', True, 'march']
    df.loc[df['band_abbrev'] == 'rush-rosemont', ['band_abbrev', 'hasbandendspacers']] = ['rush', True]
    df.loc[df['band_abbrev'] == 'rush_midamerdreams_plat_sc', ['band_abbrev', 'hasbandendspacers']] = ['rush', True]
    df.loc[df['band_abbrev'] == 'rush_streets-of-diversity_d', ['band_abbrev', 'hasbandendspacers']] = ['rush', True]
    df.loc[df['band_abbrev'] == 'rush_the-secret-touch_d', ['band_abbrev', 'hasbandendspacers']] = ['rush', True]
    df.loc[df['band_abbrev'] == 'rush_twin-meadows_cd', ['band_abbrev', 'hasbandendspacers']] = ['rush', True]
    df.loc[df['band_abbrev'] == 'rush_waltz-of-the-shreves_cd', ['band_abbrev', 'hasbandendspacers']] = ['rush', True]
    df.loc[df['band_abbrev'] == 'ry nyc', ['band_abbrev', 'hasbandendspacers']] = ['ry', True]
    df.loc[df['band_abbrev'] == 'sacred reich_denver co', ['band_abbrev', 'hasbandendspacers']] = ['sacred reich', True]
    df.loc[df['band_abbrev'] == 'santana fw', ['band_abbrev', 'hasbandendspacers']] = ['santana', True]
    df.loc[df['band_abbrev'] == 'santana knoxville', ['band_abbrev', 'hasbandendspacers']] = ['santana', True]
    df.loc[df['band_abbrev'] == 'santana-shoreline oct', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['santana', True, 'oct']
    df.loc[df['band_abbrev'] == 'saxon - october', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['saxon', True, 'october']
    df.loc[df['band_abbrev'] == 'sc-disc', ['band_abbrev', 'hasbandendspacers']] = ['sc', True]
    df.loc[df['band_abbrev'] == 'sd-disc', ['band_abbrev', 'hasbandendspacers']] = ['sd', True]
    df.loc[df['band_abbrev'] == 'second nature cd', ['band_abbrev', 'hasbandendspacers']] = ['second nature', True]
    df.loc[df['band_abbrev'] == 'secret_empire_d', ['band_abbrev', 'hasbandendspacers']] = ['secret_empire', True]
    df.loc[df['band_abbrev'] == 'set one track', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'shelby lynne  kuumbwa', ['band_abbrev', 'hasbandendspacers']] = ['shelby lynne', True]
    df.loc[df['band_abbrev'] == 'shellac - glasgow', ['band_abbrev', 'hasbandendspacers']] = ['shellac', True]
    df.loc[df['band_abbrev'] == 'sigurros_copenhagen', ['band_abbrev', 'hasbandendspacers']] = ['sigurros', True]
    df.loc[df['band_abbrev'] == 'skb-disc', ['band_abbrev', 'hasbandendspacers']] = ['skb', True]
    df.loc[df['band_abbrev'] == 'skids-holmfirth picturedrome', ['band_abbrev', 'hasbandendspacers']] = ['skids', True]
    df.loc[df['band_abbrev'] == 'slash_download', ['band_abbrev', 'hasbandendspacers']] = ['slash', True]
    df.loc[df['band_abbrev'] == 'slash_manchester', ['band_abbrev', 'hasbandendspacers']] = ['slash', True]
    df.loc[df['band_abbrev'] == 'slayer phoenix', ['band_abbrev', 'hasbandendspacers']] = ['slayer', True]
    df.loc[df['band_abbrev'] == 'slo burn_las vegas nv', ['band_abbrev', 'hasbandendspacers']] = ['slo burn', True]
    df.loc[df['band_abbrev'] == 'slowdive commodore ballroom vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['slowdive', True]
    df.loc[df['band_abbrev'] == 'soester gloria-singen', ['band_abbrev', 'hasbandendspacers']] = ['soester gloria', True]
    df.loc[df['band_abbrev'] == 'son of the velvet rat - hartberg, austria', ['band_abbrev', 'hasbandendspacers']] = ['son of the velvet rat', True]
    df.loc[df['band_abbrev'] == 'son of the velvet rat - vienna', ['band_abbrev', 'hasbandendspacers']] = ['son of the velvet rat', True]
    df.loc[df['band_abbrev'] == 'soulive-disc', ['band_abbrev', 'hasbandendspacers']] = ['soulive', True]
    df.loc[df['band_abbrev'] == 'soundcheck track', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'southern blood-celebrating gregg allman', ['band_abbrev', 'hasbandendspacers']] = ['southern blood', True]
    df.loc[df['band_abbrev'] == 'spaceman spiff - graz, austria', ['band_abbrev', 'hasbandendspacers']] = ['spaceman spiff', True]
    df.loc[df['band_abbrev'] == 'spirit berlin', ['band_abbrev', 'hasbandendspacers']] = ['spirit', True]
    df.loc[df['band_abbrev'] == 'spirit tx', ['band_abbrev', 'hasbandendspacers']] = ['spirit', True]
    df.loc[df['band_abbrev'] == 'spirit_of_baseball_d', ['band_abbrev', 'hasbandendspacers']] = ['spirit_of_baseball', True]
    df.loc[df['band_abbrev'] == 'ssj-ritz, nyc cd', ['band_abbrev', 'hasbandendspacers']] = ['ssj', True]
    df.loc[df['band_abbrev'] == 'steely dan woodlands texas sept', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['steely dan', True, 'sept']
    df.loc[df['band_abbrev'] == 'stereolab-brighton pavillion', ['band_abbrev', 'hasbandendspacers']] = ['stereolab', True]
    df.loc[df['band_abbrev'] == 'steve winwood  fox theater  oakland', ['band_abbrev', 'hasbandendspacers']] = ['steve winwood', True]
    df.loc[df['band_abbrev'] == 'sting_wembley stadium', ['band_abbrev', 'hasbandendspacers']] = ['sting', True]
    df.loc[df['band_abbrev'] == 'stockholm_d', ['band_abbrev', 'hasbandendspacers']] = ['stockholm', True]
    df.loc[df['band_abbrev'] == 'suede - reading festival', ['band_abbrev', 'hasbandendspacers']] = ['suede', True]
    df.loc[df['band_abbrev'] == 'sw malaga', ['band_abbrev', 'hasbandendspacers']] = ['sw', True]
    df.loc[df['band_abbrev'] == 't', ['band', 'hasbandendspacers']] = ['notband', True]
    df.loc[df['band_abbrev'] == 'tab-disc', ['band_abbrev', 'hasbandendspacers']] = ['tab', True]
    df.loc[df['band_abbrev'] == 'take me to the river  harris center  folsom ca', ['band_abbrev', 'hasbandendspacers']] = ['take me to the river  harris center', True]
    df.loc[df['band_abbrev'] == 'tangerine dream october', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['tangerine dream', True, 'october']
    df.loc[df['band_abbrev'] == 'terje rypdal - copenhagen', ['band_abbrev', 'hasbandendspacers']] = ['terje rypdal', True]
    df.loc[df['band_abbrev'] == 'the blow fox cabaret vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['the blow', True]
    df.loc[df['band_abbrev'] == 'the breeders - washington dc', ['band_abbrev', 'hasbandendspacers']] = ['the breeders', True]
    df.loc[df['band_abbrev'] == 'the breeders_london', ['band_abbrev', 'hasbandendspacers']] = ['the breeders', True]
    df.loc[df['band_abbrev'] == 'the courtneys biltmore cabaret vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['the courtneys', True]
    df.loc[df['band_abbrev'] == 'the cranberries - washington dc', ['band_abbrev', 'hasbandendspacers']] = ['the cranberries', True]
    df.loc[df['band_abbrev'] == 'the cure sant jordi', ['band_abbrev', 'hasbandendspacers']] = ['the cure', True]
    df.loc[df['band_abbrev'] == 'the dove and the wolf fox cabaret vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['the dove and the wolf', True]
    df.loc[df['band_abbrev'] == 'the flesh eaters rickshaw theatre vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['the flesh eaters', True]
    df.loc[df['band_abbrev'] == 'the pack ad rickshaw theatre vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['the pack', True]
    df.loc[df['band_abbrev'] == 'the soul shakers lugano', ['band_abbrev', 'hasbandendspacers']] = ['the soul shakers', True]
    df.loc[df['band_abbrev'] == 'the spectrum, montreal, quebec-d', ['band_abbrev', 'hasbandendspacers']] = ['the spectrum', True]
    df.loc[df['band_abbrev'] == 'the the casino paris', ['band_abbrev', 'hasbandendspacers']] = ['the', True]
    df.loc[df['band_abbrev'] == 'the_bayou-electric_d', ['band_abbrev', 'hasbandendspacers']] = ['the_bayou', True]
    df.loc[df['band_abbrev'] == 'the_who_seattle', ['band_abbrev', 'hasbandendspacers']] = ['the_who', True]
    df.loc[df['band_abbrev'] == 'theessink - evans_set', ['band_abbrev', 'hasbandendspacers']] = ['theessink', True]
    df.loc[df['band_abbrev'] == 'themagpiesalute.musicbox.hwllywd.ca', ['band_abbrev', 'hasbandendspacers']] = ['themagpiesalute', True]
    df.loc[df['band_abbrev'] == 'thin lizzy - live tokyo', ['band_abbrev', 'hasbandendspacers']] = ['thin lizzy', True]
    df.loc[df['band_abbrev'] == 'thunders lloyd stones jam - a side tr', ['band_abbrev', 'hasbandendspacers']] = ['thunders lloyd stones jam', True]
    df.loc[df['band_abbrev'] == 'toddla t', ['band_abbrev', 'hasbandendspacers']] = ['toddla', True]
    df.loc[df['band_abbrev'] == 'tom petty fillmore jan', ['band_abbrev', 'hasbandendspacers', 'month_in_band_abbrev']] = ['tom petty', True, 'jan']
    df.loc[df['band_abbrev'] == 'tones on tail - track', ['band_abbrev', 'hasbandendspacers']] = ['tom tones on tail', True]
    df.loc[df['band_abbrev'] == 'tori amos  theater at the ace hotel   la', ['band_abbrev', 'hasbandendspacers']] = ['tori amos', True]
    df.loc[df['band_abbrev'] == 'toronto_d', ['band_abbrev', 'hasbandendspacers']] = ['toronto', True]
    df.loc[df['band_abbrev'] == 'torres fox cabaret vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['torres', True]
    df.loc[df['band_abbrev'] == 'total immersion-leonard bernstein', ['band_abbrev', 'hasbandendspacers']] = ['leonard bernstein', True]
    df.loc[df['band_abbrev'] == 'trail of dead duisburg', ['band_abbrev', 'hasbandendspacers']] = ['trail of dead', True]
    df.loc[df['band_abbrev'] == 'treyanastasio.observatory', ['band_abbrev', 'hasbandendspacers']] = ['treyanastasio', True]
    df.loc[df['band_abbrev'] == 'turn_the_pages_d', ['band_abbrev', 'hasbandendspacers']] = ['turn_the_pages', True]
    df.loc[df['band_abbrev'] == 'tvz-villingen', ['band_abbrev', 'hasbandendspacers']] = ['tvz', True]
    df.loc[df['band_abbrev'] == 'tvz_gcfolkscene', ['band_abbrev', 'hasbandendspacers']] = ['tvz', True]
    df.loc[df['band_abbrev'] == 'tvzdutchradio', ['band_abbrev', 'hasbandendspacers']] = ['tvz', True]
    df.loc[df['band_abbrev'] == 'tvzhouston', ['band_abbrev', 'hasbandendspacers']] = ['tvz', True]
    df.loc[df['band_abbrev'] == 'tvzmarburg', ['band_abbrev', 'hasbandendspacers']] = ['tvz', True]
    df.loc[df['band_abbrev'] == 'tvzportland', ['band_abbrev', 'hasbandendspacers']] = ['tvz', True]
    df.loc[df['band_abbrev'] == 'twr camden falcon', ['band_abbrev', 'hasbandendspacers']] = ['twr', True]
    df.loc[df['band_abbrev'] == 'ufo - october', ['band_abbrev', 'hasbandendspacers','month_in_band_abbrev']] = ['roy harper jimmy page', True, 'october']
    df.loc[df['band_abbrev'] == 'uncle tupelo kdhx - acoustic - cut interviews - ut', ['band_abbrev', 'hasbandendspacers']] = ['uncle tupelo', True]
    df.loc[df['band_abbrev'] == 'van halen - april', ['band_abbrev', 'hasbandendspacers','month_in_band_abbrev']] = ['van halen', True, 'april']
    df.loc[df['band_abbrev'] == 'van halen - september', ['band_abbrev', 'hasbandendspacers','month_in_band_abbrev']] = ['van halen', True, 'september']
    df.loc[df['band_abbrev'] == 'van morrison - kansas city january', ['band_abbrev', 'hasbandendspacers','month_in_band_abbrev']] = ['van halen', True, 'january']
    df.loc[df['band_abbrev'] == 'van morrison - milwaukee may', ['band_abbrev', 'hasbandendspacers','month_in_band_abbrev']] = ['van halen', True, 'may']
    df.loc[df['band_abbrev'] == 'various - haldern pop', ['band_abbrev', 'hasbandendspacers']] = ['haldern pop', True]
    df.loc[df['band_abbrev'] == 'various pipers glenfiddich', ['band_abbrev', 'hasbandendspacers']] = ['pipers', True]
    df.loc[df['band_abbrev'] == 'vega_boston', ['band_abbrev', 'hasbandendspacers']] = ['vega', True]
    df.loc[df['band_abbrev'] == 'vh_sanfrancisco_ca', ['band_abbrev', 'hasbandendspacers']] = ['vh', True]
    df.loc[df['band_abbrev'] == 'vh_worcester_ma', ['band_abbrev', 'hasbandendspacers']] = ['vh', True]
    df.loc[df['band_abbrev'] == 'vintage trouble  oakland art & soul fest', ['band_abbrev', 'hasbandendspacers']] = ['vintage trouble', True]
    df.loc[df['band_abbrev'] == 'virgin marys_manchester', ['band_abbrev', 'hasbandendspacers']] = ['virgin marys', True]
    df.loc[df['band_abbrev'] == 'vm - copenhagen april', ['band_abbrev', 'hasbandendspacers','month_in_band_abbrev']] = ['vm', True, 'april']
    df.loc[df['band_abbrev'] == 'vm_bristol', ['band_abbrev', 'hasbandendspacers']] = ['vm', True]
    df.loc[df['band_abbrev'] == 'volbeat - parken, copenhagen', ['band_abbrev', 'hasbandendspacers']] = ['volbeat', True]
    df.loc[df['band_abbrev'] == 'warpaint live at vida', ['band_abbrev', 'hasbandendspacers']] = ['warpaint', True]
    df.loc[df['band_abbrev'] == 'waterboys birmingham', ['band_abbrev', 'hasbandendspacers']] = ['waterboys', True]
    df.loc[df['band_abbrev'] == 'waterboys palladium', ['band_abbrev', 'hasbandendspacers']] = ['waterboys', True]
    df.loc[df['band_abbrev'] == 'waterboys toronto', ['band_abbrev', 'hasbandendspacers']] = ['waterboys', True]
    df.loc[df['band_abbrev'] == 'waterboys_galway', ['band_abbrev', 'hasbandendspacers']] = ['waterboys', True]
    df.loc[df['band_abbrev'] == 'waxahatchee the imperial vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['waxahatchee', True]
    df.loc[df['band_abbrev'] == 'whiskey gentry merry yalltide celebration compilation xxxx', ['band_abbrev', 'hasbandendspacers']] = ['whiskey gentry', True]
    df.loc[df['band_abbrev'] == 'winter the cobalt vancouver bc', ['band_abbrev', 'hasbandendspacers']] = ['winter', True]
    df.loc[df['band_abbrev'] == 'wounded lion in the jungle long rehearsal(gd', ['band_abbrev', 'hasbandendspacers']] = ['gd', True]
    df.loc[df['band_abbrev'] == 'wsp-disc', ['band_abbrev', 'hasbandendspacers']] = ['wsp', True]
    df.loc[df['band_abbrev'] == 'x_joseph_arthur', ['band_abbrev', 'hasbandendspacers']] = ['joseph_arthur', True]
    df.loc[df['band_abbrev'] == 'yes-arw', ['band_abbrev', 'hasbandendspacers']] = ['yes', True]
    df.loc[df['band_abbrev'] == 'zen arcade outtakes - d', ['band_abbrev', 'hasbandendspacers']] = ['zen arcade', True]

    return df

print('created band-fixing function')


In [ ]:
# GET LIST OF OFFICIAL ABBREVIATIONS

print('Retrieving from etree')
pageurl = 'http://wiki.etree.org/index.php?page=BandAbbreviations'
page = urllib.request.urlopen(pageurl).read()
soup = bs(page, "html.parser")
tables = soup.find_all('table')

#for thistable in tables:
#    print(thistable)
#    print('---------')

bandtable = tables[1]
bands = bandtable.find_all('tr')
allbands = []

print('Thinking...')
for thisband in bands[1:]:
    theirdata = thisband.find_all('td')
    oneband = {}
    oneband['name'] = theirdata[0].text
    oneband['abbrev'] = theirdata[1].text
    try:
        oneband['otherabbrev'] = theirdata[2].text
    except:
        pass
    try:
        oneband['notes'] = theirdata[3].text
    except:
        pass
    allbands.append(oneband)
abbrev_df = pandas.DataFrame.from_records(allbands)
abbrev_df.index.name = 'rownumber'

abbrev_df = abbrev_df.drop('notes', axis=1)
column_order = ['name', 'abbrev', 'otherabbrev']
abbrev_df = abbrev_df[column_order]

# Trim one space from beginning and end of band abbrev and name
abbrev_df['abbrev'] = abbrev_df['abbrev'].apply(lambda x: str(x).strip())
abbrev_df['name'] = abbrev_df['name'].apply(lambda x: str(x).strip())

# Trim away brackets from beginning and end of band name (for some bands)
abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[0])) == '['), 
                             abbrev_df['name'].apply(lambda x: str(x)[1:]), 
                             abbrev_df['name']) 
abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[len(str(x))-1])) == ']'),
                              abbrev_df['name'].apply(lambda x: str(x)[:-1]), 
                              abbrev_df['name'])
# dumb special cases
abbrev_df.loc[abbrev_df['abbrev'] == '(see FB below)', 'abbrev'] = ''
abbrev_df.loc[abbrev_df['abbrev'] == 'Keller Williams w- String Cheese Incident', 'abbrev'] = 'kwi'


# lower and strip, fillna w/blanks for otherabbrev
abbrev_df['abbrev'] = abbrev_df['abbrev'].apply(lambda x: str(x).lower().strip())
abbrev_df['otherabbrev'] = abbrev_df['otherabbrev'].fillna('')
abbrev_df['otherabbrev'] = abbrev_df['otherabbrev'].apply(lambda x: str(x).lower().strip())

abbrev_df_bk = abbrev_df
print('ok')
abbrev_df.sample(3)

In [ ]:
# ROTATE ABBREVIATIONS INTO GUIDE FILE
guide_df = pandas.DataFrame(data=abbrev_df['abbrev'][abbrev_df['abbrev'] != ''], columns=['abbrev'])

guide_df = guide_df.merge(abbrev_df, how='left', on='abbrev')
guide_df = guide_df[['abbrev', 'name']]
guide_df = guide_df.set_index('abbrev')
guide_df['canonical'] = True

variant_abbrev_list = []
for thisrow in abbrev_df['otherabbrev'][abbrev_df['otherabbrev'] != ''].drop_duplicates().tolist():
    for thisabbrev in thisrow.split(','):
        thisabbrev = thisabbrev.strip()
        thisband = abbrev_df['name'][abbrev_df['otherabbrev'].apply(lambda x: thisabbrev in x)].values[0]
        if thisabbrev not in guide_df.index.values:
            guide_df.loc[thisabbrev] = [thisband, False]

guide_df = guide_df.sort_values('canonical').sort_index()

# change index value of kdub
indexaslist = guide_df.index.tolist()
kdubvalue = indexaslist.index('kdub (for k-double-u)')
indexaslist[kdubvalue] = 'kdub'
guide_df.index = indexaslist

# Clean up some weird-ass values
guide_df.loc['beanland', 'name'] = 'Beanland'
guide_df.loc['buho', 'canonical'] = True
guide_df.loc['logic', 'canonical'] = True
guide_df.loc['gtb', 'canonical'] = True
guide_df.loc['ig', 'canonical'] = True
guide_df.loc['jgb', 'name'] = 'Jerry Garcia Band'
guide_df.loc['jgb', 'canonical'] = True
guide_df.loc['jt', 'canonical'] = True
guide_df.loc['ls', 'canonical'] = True
guide_df.loc['los', 'canonical'] = True
guide_df.loc['marlow', 'canonical'] = True
guide_df.loc['moe', 'name'] = 'Moe'
guide_df.loc['pmb', 'canonical'] = True
guide_df.loc['amendola', 'canonical'] = True
guide_df.loc['hip', 'canonical'] = True
guide_df.loc['wilco', 'canonical'] = True
guide_df.loc['spod', 'name'] = 'Serial Pod'

guide_df.index.name = 'abbrev'

guide_df_bk = guide_df
guide_df.sample(3)

In [ ]:
#from pprint import pprint

print('Retrieving from archive.org...')

pageurl = 'https://archive.org/audio/etree-band-abbrevs.php'

    
page = urllib.request.urlopen(pageurl).read()
soup = bs(page, "html.parser")

tables = soup.find_all('table')

bandtable = tables[0]
bands = bandtable.find_all('tr')
allbands = []

print('Thinking...')
for thisband in bands[2:]:
    #pprint(thisband)
    theirdata = thisband.find_all('td')
    #pprint(theirdata)
    oneband = {}
    oneband['name'] = theirdata[0].text
    oneband['abbrevs'] = theirdata[1].text
    allbands.append(oneband)

archive_org_abbrevs_df = pandas.DataFrame.from_records(allbands)

archive_org_abbrevs_df = archive_org_abbrevs_df.set_index('name')

archive_org_abbrevs_df = archive_org_abbrevs_df.assign(abbrev_list = '')
for idx, thisrow in archive_org_abbrevs_df.iterrows():
    archive_org_abbrevs_df.loc[idx, 'abbrev_list'] = archive_org_abbrevs_df['abbrevs'].loc[idx].split(';')

allabbrevdict = {}
for idx, thisrow in archive_org_abbrevs_df.iterrows():
    for i in range(0, len(thisrow['abbrev_list'])):
        allabbrevdict[thisrow['abbrev_list'][i]] = idx
#allabbrevdict
archive_org_guide_df = pandas.DataFrame(list(allabbrevdict.items()), columns=['abbrev', 'band'])
archive_org_guide_df = archive_org_guide_df.set_index('abbrev')

archive_org_guide_df_bk = archive_org_guide_df
archive_org_guide_df.sample(3)

In [ ]:
music_df = music_df_bk

music_df['abbrev_type'] = 'notfound'
music_df.loc[music_df['band'] == 'blank', 'abbrev_type'] = 'blank'
music_df.loc[music_df['band'] == 'notband', 'abbrev_type'] = 'notband'

music_df = music_df.reset_index().join(guide_df, how='left', on='band_abbrev').set_index('ID')

music_df.loc[music_df['name'].notnull(), 'band'] = music_df['name']

music_df = music_df.drop('name', axis=1)

music_df.loc[music_df['canonical'] == True, 'abbrev_type'] = 'canonical'
music_df.loc[music_df['canonical'] == False, 'abbrev_type'] = 'variant'

incorrect = re.compile('(phish|pl&f|gov|deadfeat)+')

music_df.loc[music_df['band_abbrev'].apply(lambda x: re.search(incorrect, x)).notnull(), 'abbrev_type'] = 'incorrect'

print(music_df.groupby('abbrev_type').size())

#music_df[['band_abbrev', 'band', 'abbrev_type']][(music_df['abbrev_type'] == 'variant')].drop_duplicates().sort_values(by='band_abbrev')
music_df_bk = music_df

In [ ]:
music_df = music_df_bk
archive_org_guide_df = archive_org_guide_df_bk

archive_org_guide_df = archive_org_guide_df.rename(columns={'band': 'name'})
music_df = music_df.reset_index().join(archive_org_guide_df, how='left', on='band_abbrev').set_index('ID')

music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['name'].notnull()), ['band', 'abbrev_type']] = [music_df['name'], 'additional']

music_df = music_df.drop('name', axis=1)

music_df_bk = music_df

print(music_df.groupby('abbrev_type').size())

In [ ]:
print('reading from backup...')
music_df = music_df_bk

bands_df = pandas.read_csv('band_results_jeremy.csv', encoding='utf-8')
bands_df = bands_df.set_index('band_abbrev')
bands_df.loc[bands_df['date_right_after_band'] == 'FALSE', 'date_right_after_band'] = False
bands_df['date_right_after_band'] = bands_df['date_right_after_band'].fillna(True)

# remove quotes
bands_df['band'] = bands_df['band'].apply(lambda x: x[1:-1])
bands_df['notes'] = bands_df['notes'].apply(lambda x: x[1:-1])
#music_df['band_abbrev'].sample(3)

# rename column
bands_df = bands_df.rename(columns={'band': 'band_jeremy'})

music_df = music_df.merge(bands_df, how='left', on='band_abbrev')

music_df.loc[(music_df['band'] == '') & (music_df['band_jeremy'].notnull()), 'band'] = music_df['band_jeremy']
music_df.loc[music_df['band_jeremy'].notnull(), 'abbrev_type'] = 'jeremy'

music_df[['band_abbrev', 'band', 'abbrev_type']].sample(20)
music_df.groupby('abbrev_type').size()

music_df.loc[
    (music_df['abbrev_type'] == 'notfound') & 
    (music_df['band_abbrev'].apply(lambda x: 'igu' in x)), ['band', 'abbrev_type']
] = ['Sigur Ros', 'jeremy']
music_df.loc[
    (music_df['abbrev_type'] == 'notfound') & 
    (music_df['band_abbrev'].apply(lambda x: 'peter' in x)), ['band', 'abbrev_type']
] = ['Peter Brötzmann', 'jeremy']
music_df.loc[
    (music_df['abbrev_type'] == 'notfound') & 
    (music_df['band_abbrev'].apply(lambda x: 'nils' in x)), ['band', 'abbrev_type']
] = ['Nils Petter Molvær', 'jeremy']
music_df.loc[
    (music_df['abbrev_type'] == 'notfound') & 
    (music_df['band_abbrev'].apply(lambda x: 'yra' in x)), ['band', 'abbrev_type']
] = ['Myrath', 'jeremy']
music_df.loc[
    (music_df['abbrev_type'] == 'notfound') & 
    (music_df['band_abbrev'].apply(lambda x: 'meth' in x)), ['band', 'abbrev_type']
] = ['Pat Metheny', 'incorrect']
music_df.loc[
    (music_df['abbrev_type'] == 'notfound') & 
    (music_df['band_abbrev'].apply(lambda x: 'uur' in x)), ['band', 'abbrev_type']
] = ['Vuur', 'jeremy']
music_df.loc[
    (music_df['abbrev_type'] == 'notfound') & 
    (music_df['band_abbrev'].apply(lambda x: 'uur' in x)), ['band', 'abbrev_type']
] = ['Fräulein Hona', 'jeremy']
music_df.loc[
    (music_df['abbrev_type'] == 'notfound'), ['band', 'abbrev_type']
] = ['', 'notband']
music_df.loc[(music_df['abbrev_type'] == 'jeremy') & (music_df['band'] == 'notfound'), 
             'abbrev_type'] = 'notband'


music_df.loc[(music_df['abbrev_type'] == 'jeremy') & (music_df['band'] == 'incorrect'), 'abbrev_type'] = 'incorrect'
music_df.loc[(music_df['abbrev_type'] == 'jeremy') & (music_df['band'] == 'notband'), 'abbrev_type'] = 'notband'


print('backing up...')
music_df_bk = music_df

print(music_df.groupby('abbrev_type').size())


In [ ]:
music_df = music_df_bk
datefinder = re.compile('[\d]+[\s\-\._]+[\d]+[\s\-\._]+[\d]+')
yyyyfinder = re.compile('[\d]{4}-[\d]{2}-[\d]{2}')
yyfinder = re.compile('[\d]{2}-[\d]{2}-[\d]{2}')
date1 = re.compile('[\d]{2,4}[\s\.\-_]*[\d]{1,2}[\s\.\-_]*[\d]{1,2}')
date2 = re.compile('^[\d]{6,8}')

# initialize
music_df['thedate'] = pandas.to_datetime(np.nan)
music_df['mics_etc'] = np.nan
music_df = music_df.assign(date_format='otros')

# parse for yyyy-mm-dd date values
print('parsing dates as yyyy-mm-dd...')
music_df.loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(yyyyfinder, x)
        ).notnull(), 'thedate'
    ] = pandas.to_datetime(music_df['postbandstuff'].loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(yyyyfinder, x)
        ).notnull()
    ].apply(lambda x: 
            x[re.search(yyyyfinder, x).start():re.search(yyyyfinder, x).end()]),
                    errors='coerce')
music_df.loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(yyyyfinder, x)
        ).notnull(), 'mics_etc'
    ] = music_df['postbandstuff'].loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(yyyyfinder, x)
        ).notnull()
    ].apply(lambda x: 
            x[re.search(yyyyfinder, x).end():len(x)])
music_df.loc[
    (music_df['date_format'] == 'otros') & 
    (music_df['thedate'].notnull()), 'date_format'] = 'yyyy-mm-dd'

# parse for yy-mm-dd date values
print('parsing dates as yy-mm-dd...')
music_df.loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(yyfinder, x)
        ).notnull(), 'thedate'
    ] = pandas.to_datetime(music_df['postbandstuff'].loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(yyfinder, x)
        ).notnull()
    ].apply(lambda x: 
            x[re.search(yyfinder, x).start():re.search(yyfinder, x).end()]),
                    errors='coerce')
music_df.loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(yyfinder, x)
        ).notnull(), 'mics_etc'
    ] = music_df['postbandstuff'].loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(yyfinder, x)
        ).notnull()
    ].apply(lambda x: 
            x[re.search(yyfinder, x).end():len(x)])
music_df.loc[
    (music_df['date_format'] == 'otros') & 
    (music_df['thedate'].notnull()), 'date_format'] = 'yy-mm-dd'

# parse for date1 date values
print('parsing dates as date1 regex format...')
music_df.loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(date1, x)
        ).notnull(), 'thedate'
    ] = pandas.to_datetime(music_df['postbandstuff'].loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(date1, x)
        ).notnull()
    ].apply(lambda x: 
            x[re.search(date1, x).start():re.search(date1, x).end()]),
                    errors='coerce')
music_df.loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(date1, x)
        ).notnull(), 'mics_etc'
    ] = music_df['postbandstuff'].loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(date1, x)
        ).notnull()
    ].apply(lambda x: 
            x[re.search(date1, x).end():len(x)])
music_df.loc[
    (music_df['date_format'] == 'otros') & 
    (music_df['thedate'].notnull()), 'date_format'] = 'date1'


# parse for date2 date values
print('parsing dates as date2 regex format...')
music_df.loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(date2, x)
        ).notnull(), 'thedate'
    ] = pandas.to_datetime(music_df['postbandstuff'].loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(date2, x)
        ).notnull()
    ].apply(lambda x: 
            x[re.search(date2, x).start():re.search(date2, x).end()]),
                    errors='coerce')
music_df.loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(date2, x)
        ).notnull(), 'mics_etc'
    ] = music_df['postbandstuff'].loc[
        music_df['postbandstuff'].apply(
            lambda x: re.search(date2, x)
        ).notnull()
    ].apply(lambda x: 
            x[re.search(date2, x).end():len(x)])
music_df.loc[
    (music_df['date_format'] == 'otros') & 
    (music_df['thedate'].notnull()), 'date_format'] = 'date2'

# Set up year and month for filling in by hand
music_df['theyear'] = np.nan
music_df['themonth'] = np.nan
music_df['theday'] = np.nan

# By hand (both full dates and date parts)
print('fixing dates by hand...')
music_df = fixdates(music_df)

#music_df[['postbandstuff', 'thedate', 'mics_etc', 'date_format']].sample(20)#[music_df['thedate'].notnull()].sample(20)

print('backing up...')
music_df_bk = music_df

music_df.groupby('date_format').size()

In [ ]:
datefinder = re.compile('[\d]+[\s\-\._]+[\d]+[\s\-\._]+[\d]+')


music_df[['postbandstuff']][
    (music_df['date_format'] == 'otros') & 
    (music_df['postbandstuff'] != '') & 
    (music_df['postbandstuff'].apply(lambda x: re.search(datefinder, x)).notnull())
].drop_duplicates()#.sort_index().values.tolist()#.sort_values(by='postbandstuff')#[25:50]


In [ ]:
def fixdates(df):
    music_df.loc[24, ['thedate', 'mics_etc', 'date_format']] = ['1989-09-29', '_hamburg_track_16', 'byhand']
    music_df.loc[31, ['thedate', 'mics_etc', 'date_format']] = ['1992-09-07', '_braunschweig_track_12', 'byhand']
    music_df.loc[75, ['thedate', 'mics_etc', 'date_format']] = ['1977-06-02', ' D01T01', 'byhand']
    music_df.loc[141, ['thedate', 'mics_etc', 'date_format']] = ['1991-10-24', ' _NJ05', 'byhand']
    music_df.loc[172, ['thedate', 'mics_etc', 'date_format']] = ['1981-05-11', ' _ann arbor_track_11', 'byhand']
    music_df.loc[190, ['thedate', 'mics_etc', 'date_format']] = ['2017-08-19', ' ccm4s RX-AA-final01', 'byhand']
    music_df.loc[222, ['thedate', 'mics_etc', 'date_format']] = ['1984-07-01', ' _fairfax_track_14', 'byhand']
    music_df.loc[245, ['thedate', 'mics_etc', 'date_format']] = ['1989-02-02', '01', 'byhand']
    music_df.loc[295, ['thedate', 'mics_etc', 'date_format']] = ['1983-11-30', '_mannheim_track_24', 'byhand']
    music_df.loc[405, ['thedate', 'mics_etc', 'date_format']] = ['2013-08-13', ' ccm4s DIME edit EQ-final01', 'byhand']    
    music_df.loc[535, ['thedate', 'mics_etc', 'date_format']] = ['1983-11-19', '_neunkirchen_track_24', 'byhand']    
    music_df.loc[557, ['thedate', 'mics_etc', 'date_format']] = ['2017-08-17', ' 101', 'byhand']
    music_df.loc[564, ['thedate', 'mics_etc', 'date_format']] = ['1980-09-06', ' _goeteborg_track_18', 'byhand']
    music_df.loc[613, ['thedate', 'mics_etc', 'date_format']] = ['2017-09-02', ' ccm4s set one RX-EQ-201', 'byhand']
    music_df.loc[682, ['thedate', 'mics_etc', 'date_format']] = ['2017-09-17', ' ccm4s 24-48 RXEQ-final01', 'byhand']
    music_df.loc[705, ['thedate', 'mics_etc', 'date_format']] = ['1983-02-28', '_hamburg_track_06', 'byhand']
    music_df.loc[914, ['thedate', 'mics_etc', 'date_format']] = ['2017-09-07', ' Oz Noy 01', 'byhand']
    music_df.loc[966, ['thedate', 'mics_etc', 'date_format']] = ['1981-08-16', '_rotterdam_track_12', 'byhand']
    music_df.loc[968, ['thedate', 'mics_etc', 'date_format']] = ['1981-08-16', '_rotterdam_track_01', 'byhand']
    music_df.loc[974, ['thedate', 'mics_etc', 'date_format']] = ['1987-11-04', '_frankfurt_track_15', 'byhand']    
    music_df.loc[993, ['thedate', 'mics_etc', 'date_format']] = ['1985-04-24', '_syracuse_track_14', 'byhand']    
    music_df.loc[1002, ['thedate', 'mics_etc', 'date_format']] = ['1988-04-23', '_frankfurt_track_16', 'byhand']        
    music_df.loc[1009, ['thedate', 'mics_etc', 'date_format']] = ['1990-09-02', '_ulm_track_09', 'byhand']
    music_df.loc[1034, ['thedate', 'mics_etc', 'date_format']] = ['1982-02-11', '_hamburg_track_20', 'byhand']   
    music_df.loc[1070, ['thedate', 'mics_etc', 'date_format']] = ['2017-03-24', '-220519 set 1 Soweiso Schlippenbach Mahall Tom', 'byhand']
    music_df.loc[1078, ['thedate', 'mics_etc', 'date_format']] = ['1988-03-02', '_mannheim_track_15', 'byhand']
    music_df.loc[1092, ['thedate', 'mics_etc', 'date_format']] = ['1979-02-01', ' (complete) d2 - 04', 'byhand']
    music_df.loc[1113, ['thedate', 'mics_etc', 'date_format']] = ['2000-12-04', '_heidelberg_track_11', 'byhand']
    music_df.loc[1154, ['thedate', 'mics_etc', 'date_format']] = ['2016-08-14', ' ccm4s-working RX-EQ01', 'byhand']
    music_df.loc[1159, ['thedate', 'mics_etc', 'date_format']] = ['1988-02-05', ' _philadelphia_track_06', 'byhand']
    music_df.loc[1168, ['thedate', 'mics_etc', 'date_format']] = ['1995-09-29', ' _brussels_track_09', 'byhand']
    music_df.loc[1182, ['thedate', 'mics_etc', 'date_format']] = ['2011-07-21', ' 01', 'byhand']
    music_df.loc[1188, ['thedate', 'mics_etc', 'date_format']] = ['2011-07-21', ' 01', 'byhand']
    music_df.loc[1212, ['thedate', 'mics_etc', 'date_format']] = ['1979-09-12', ' _Rush_UtA79M_D1T01', 'byhand']
    music_df.loc[1239, ['thedate', 'mics_etc', 'date_format']] = ['1982-04-07', ' -d1t01', 'byhand']    
    music_df.loc[1281, ['thedate', 'mics_etc', 'date_format']] = ['2017-02-25', ' 24-48 RX-EQ-final01', 'byhand']  
    music_df.loc[1305, ['thedate', 'mics_etc', 'date_format']] = ['1983-09-22', '_RiaN_D1T01', 'byhand']
    music_df.loc[1326, ['thedate', 'mics_etc', 'date_format']] = ['1988-02-17', '_heidelberg_track_21', 'byhand']
    music_df.loc[1331, ['thedate', 'mics_etc', 'date_format']] = ['1986-05-20', '_frankfurt_ track_09', 'byhand']
    music_df.loc[1347, ['thedate', 'mics_etc', 'date_format']] = ['1995-06-10', '_frankfurt_ track_12', 'byhand']
    music_df.loc[1375, ['thedate', 'mics_etc', 'date_format']] = ['1987-07-08', '_hannover_track_03', 'byhand']
    music_df.loc[1384, ['thedate', 'mics_etc', 'date_format']] = ['1983-05-07', '_Saarbrücken_track_07', 'byhand']
    music_df.loc[1387, ['thedate', 'mics_etc', 'date_format']] = ['1985-03-14', '_mannheim_track_12', 'byhand']
    music_df.loc[1406, ['thedate', 'mics_etc', 'date_format']] = ['1992-04-04', '_karlsruhe_track_08', 'byhand']
    music_df.loc[1419, ['thedate', 'mics_etc', 'date_format']] = ['1988-09-05', '_Paris_08', 'byhand']
    music_df.loc[1436, ['thedate', 'mics_etc', 'date_format']] = ['1989-11-08', '_heidelberg_track_08', 'byhand']
    music_df.loc[1442, ['thedate', 'mics_etc', 'date_format']] = ['2002-11-27', '_mannheim_track_11', 'byhand']
    music_df.loc[1460, ['thedate', 'mics_etc', 'date_format']] = ['2017-09-22', '_M10_4060_16bit_44K_T01', 'byhand']
    music_df.loc[1470, ['thedate', 'mics_etc', 'date_format']] = ['2011-07-23', '01', 'byhand']
    music_df.loc[1496, ['thedate', 'mics_etc', 'date_format']] = ['1969-02-14', '03', 'byhand']
    music_df.loc[1706, ['thedate', 'mics_etc', 'date_format']] = ['2017-09-30', '_M10_4060_24bit_96k_T01', 'byhand']
    music_df.loc[1707, ['thedate', 'mics_etc', 'date_format']] = ['2017-09-30', '_M10_4060_16bit_44k_T01', 'byhand']
    music_df.loc[1724, ['thedate', 'mics_etc', 'date_format']] = ['2001-11-23', '  The Necks  Jazzfest Eberswalde (Ger)', 'byhand']
    music_df.loc[1728, ['thedate', 'mics_etc', 'date_format']] = ['2002-07-21', '  Broetzmann Bauer Duo  Track 1 Open Air', 'byhand']
    music_df.loc[1748, ['thedate', 'mics_etc', 'date_format']] = ['2017-09-30', '_M10_4060_24bit_96k_T01', 'byhand']
    music_df.loc[1762, ['thedate', 'mics_etc', 'date_format']] = ['2005-03-25', 'd1tr01', 'byhand']
    music_df.loc[1865, ['thedate', 'mics_etc', 'date_format']] = ['2017-09-27', '_M10_4060_24bit_96K_T01', 'byhand']
    music_df.loc[2141, ['thedate', 'mics_etc', 'date_format']] = ['2007-07-25', 'D1Tr08', 'byhand']
    music_df.loc[2525, ['thedate', 'mics_etc', 'date_format']] = ['2008-05-07', '101 vh You really Got Me', 'byhand']
    music_df.loc[2528, ['thedate', 'mics_etc', 'date_format']] = ['2008-05-30', '101 VH warm up', 'byhand']
    music_df.loc[2645, ['thedate', 'mics_etc', 'date_format']] = ['2017-10-19', '075_01_Intro', 'byhand']
    music_df.loc[3003, ['thedate', 'mics_etc', 'date_format']] = ['2017-10-28', '_schorndorf_track_15', 'byhand']
    music_df.loc[3016, ['thedate', 'mics_etc', 'date_format']] = ['2017-10-28', '_schorndorf_track_17', 'byhand']
    music_df.loc[3063, ['thedate', 'mics_etc', 'date_format']] = ['2017-10-12', '_bensheim_matrix_18', 'byhand']
    music_df.loc[3362, ['thedate', 'mics_etc', 'date_format']] = ['2017-10-22', '_heidelberg_track_03', 'byhand']
    music_df.loc[3429, ['thedate', 'mics_etc', 'date_format']] = ['1985-09-25', '_Manchester_05', 'byhand']
    music_df.loc[3431, ['thedate', 'mics_etc', 'date_format']] = ['1987-10-16', '_Providence_18', 'byhand']
    music_df.loc[3518, ['thedate', 'mics_etc', 'date_format']] = ['1984-11-05', 'd1t01', 'byhand']
    music_df.loc[3554, ['thedate', 'mics_etc', 'date_format']] = ['2017-11-03', '  SOWIESO A von Schlippenbach & Dag Magnus Narvesen  DUO  SET 1', 'byhand']
    music_df.loc[4008, ['thedate', 'mics_etc', 'date_format']] = ['1989-02-05', ' Track01', 'byhand']
    music_df.loc[4009, ['thedate', 'mics_etc', 'date_format']] = ['1988-02-10', '_New_York_01', 'byhand']
    music_df.loc[4033, ['thedate', 'mics_etc', 'date_format']] = ['1979-04-06', '_saarbruecken_track_17', 'byhand']
    music_df.loc[4090, ['thedate', 'mics_etc', 'date_format']] = ['2017-02-16', ' Nellns MQR Version 0205', 'byhand']
    music_df.loc[4124, ['thedate', 'mics_etc', 'date_format']] = ['1980-03-04', '_hamburg_track_02', 'byhand']
    music_df.loc[4291, ['thedate', 'mics_etc', 'date_format']] = ['2017-11-07', '101-Dyin\' to Know -Joanne Shaw Taylor-The Sage Gateshead 2017-0-11-07', 'byhand']
    music_df.loc[4405, ['thedate', 'mics_etc', 'date_format']] = ['2012-09-18', ' US RAILS Madrid 0101', 'byhand']
    music_df.loc[4494, ['thedate', 'mics_etc', 'date_format']] = ['2009-08-15', '01 Dee Dee Sharp New Orleans', 'byhand']
    music_df.loc[4581, ['thedate', 'mics_etc', 'date_format']] = ['2017-11-10', '080_01_Waste_In_Space', 'byhand']
    music_df.loc[4667, ['thedate', 'mics_etc', 'date_format']] = ['2017-10-27', '076_01_Tourmente_Et_Passion', 'byhand']
    music_df.loc[4723, ['thedate', 'mics_etc', 'date_format']] = ['2009-04-05', '01', 'byhand']
    music_df.loc[4725, ['thedate', 'mics_etc', 'date_format']] = ['2004-08-25', '_d1t1', 'byhand']
    music_df.loc[4737, ['thedate', 'mics_etc', 'date_format']] = ['1989-11-26', '1_Frank_Erwin_Center_-01-Track1', 'byhand']
    music_df.loc[4762, ['thedate', 'mics_etc', 'date_format']] = ['2005-07-19', ' CD1TR01', 'byhand']
    music_df.loc[4781, ['thedate', 'mics_etc', 'date_format']] = ['1985-01-06', ' LATE11', 'byhand']
    music_df.loc[4825, ['thedate', 'mics_etc', 'date_format']] = ['2008-11-19', ' CD1TR01', 'byhand']
    music_df.loc[4933, ['thedate', 'mics_etc', 'date_format']] = ['2017-11-25', ' _hannover_track_22', 'byhand']
    music_df.loc[5203, ['thedate', 'mics_etc', 'date_format']] = ['2004-06-25', ' CD1TR01', 'byhand']
    music_df.loc[5238, ['thedate', 'mics_etc', 'date_format']] = ['2004-06-23', ' CD1TR01', 'byhand']
    music_df.loc[5248, ['thedate', 'mics_etc', 'date_format']] = ['2017-11-20', ' The Waterboys Madrid 0101', 'byhand']
    music_df.loc[5406, ['thedate', 'mics_etc', 'date_format']] = [np.nan, '01. Announcements', 'byhand']
    music_df.loc[5674, ['thedate', 'mics_etc', 'date_format']] = ['2008-07-21', ' Lou Reed Malaga 0101', 'byhand']
    music_df.loc[5691, ['thedate', 'mics_etc', 'date_format']] = ['2017-04-27', ' Paul Carrack Madrid 0101', 'byhand']
    music_df.loc[5917, ['thedate', 'mics_etc', 'date_format']] = ['1984-06-30', '-A01', 'byhand']
    music_df.loc[5958, ['thedate', 'mics_etc', 'date_format']] = ['2009-06-04', ' CD1TR01', 'byhand']
    music_df.loc[5960, ['thedate', 'mics_etc', 'date_format']] = ['2009-06-05', ' CD1TR01', 'byhand']
    music_df.loc[5962, ['thedate', 'mics_etc', 'date_format']] = ['2009-06-06', ' CD1TR01', 'byhand']
    music_df.loc[6030, ['thedate', 'mics_etc', 'date_format']] = ['1971-04-27', '101', 'byhand']
    music_df.loc[6121, ['thedate', 'mics_etc', 'date_format']] = ['2008-11-05', '_d1t1', 'byhand']
    music_df.loc[6230, ['thedate', 'mics_etc', 'date_format']] = ['2013-02-08', ' ccm4s 24-48 EQ-final01', 'byhand']
    music_df.loc[6329, ['thedate', 'mics_etc', 'date_format']] = ['2017-04-29', '072__01_L_Ecume_des_Chouans', 'byhand']
    music_df.loc[6332, ['thedate', 'mics_etc', 'date_format']] = ['2017-09-16', '074__01_Romantisme_Noir', 'byhand']
    music_df.loc[6350, ['thedate', 'mics_etc', 'date_format']] = ['2017-10-28', '079__01_La_Fabrique_Du_Fataliste', 'byhand']
    music_df.loc[6398, ['thedate', 'mics_etc', 'date_format']] = ['2017-12-10', '081__01_Introduction', 'byhand']
    music_df.loc[6478, ['thedate', 'mics_etc', 'date_format']] = ['2007-08-27', ' CD101', 'byhand']
    music_df.loc[6480, ['thedate', 'mics_etc', 'date_format']] = ['2017-10-28', '078__01_Medieval_Rites', 'byhand']
    music_df.loc[6487, ['thedate', 'mics_etc', 'date_format']] = ['2017-10-27', '077__01_Macabre_Ceremony', 'byhand']
    music_df.loc[6609, ['thedate', 'mics_etc', 'date_format']] = ['1976-10-09', '_mkc_remaster_track_09', 'byhand']
    music_df.loc[6753, ['thedate', 'mics_etc', 'date_format']] = ['2017-12-13', '_S1t01', 'byhand']
    music_df.loc[6806, ['thedate', 'mics_etc', 'date_format']] = ['1978-09-03', '_D1_T01-Intro', 'byhand']
    music_df.loc[6872, ['thedate', 'mics_etc', 'date_format']] = ['2006-11-06', '_D2_T05 - Pay Me My Money Down', 'byhand']
    music_df.loc[6961, ['thedate', 'mics_etc', 'date_format']] = ['1978-05-04', '_cbgb_matrix_track_16', 'byhand']
    music_df.loc[6970, ['thedate', 'mics_etc', 'date_format']] = ['1978-11-12', '_ny_matrix_track 23', 'byhand']
    music_df.loc[7029, ['thedate', 'mics_etc', 'date_format']] = ['1995-02-12', ' 0001', 'byhand']
    music_df.loc[7032, ['thedate', 'mics_etc', 'date_format']] = ['1997-03-23', ' 0001', 'byhand']
    music_df.loc[7575, ['thedate', 'mics_etc', 'date_format']] = ['2000-06-29', 'bd1t01', 'byhand']
    music_df.loc[7579, ['thedate', 'mics_etc', 'date_format']] = ['1981-08-02', '_ny_track_07', 'byhand']
    music_df.loc[7583, ['thedate', 'mics_etc', 'date_format']] = ['1981-11-11', '_houston_track_14', 'byhand']
    music_df.loc[7585, ['thedate', 'mics_etc', 'date_format']] = ['1981-11-13', '_austin_track_08', 'byhand']
    music_df.loc[7675, ['thedate', 'mics_etc', 'date_format']] = ['1982-10-13', '_trenton_track_14', 'byhand']
    music_df.loc[7723, ['thedate', 'mics_etc', 'date_format']] = ['1982-10-20', '_boston_track_14', 'byhand']
    music_df.loc[7773, ['thedate', 'mics_etc', 'date_format']] = ['2009-04-07', '_D2_T08-Rosalita', 'byhand']
    music_df.loc[7793, ['thedate', 'mics_etc', 'date_format']] = ['2012-07-27', ' - LC Pavillion ColumbusTrack101-fixed', 'byhand']
    music_df.loc[7811, ['thedate', 'mics_etc', 'date_format']] = ['1982-11-24', '_san diego_track_22', 'byhand']
    music_df.loc[7850, ['thedate', 'mics_etc', 'date_format']] = ['1983-03-13', '_cincinatti_track_19', 'byhand']
    music_df.loc[7953, ['thedate', 'mics_etc', 'date_format']] = ['1983-06-19', '_osaka_track_10', 'byhand']
    music_df.loc[7977, ['thedate', 'mics_etc', 'date_format']] = ['1983-07-02', '_melbourne_track_06', 'byhand']
    music_df.loc[8009, ['thedate', 'mics_etc', 'date_format']] = ['2011-05-17', 'Track101-fixed', 'byhand']
    music_df.loc[8044, ['thedate', 'mics_etc', 'date_format']] = ['1987-04-22', '_tokyo_track_08', 'byhand']
    music_df.loc[8060, ['thedate', 'mics_etc', 'date_format']] = ['1998-01-09', 'Track101', 'byhand']    
    music_df.loc[8197, ['thedate', 'mics_etc', 'date_format']] = ['2018-01-07', '_t01', 'byhand']    
    music_df.loc[8283, ['thedate', 'mics_etc', 'date_format']] = ['2011-05-06', '101Wishbone Ash, Chicago Early', 'byhand']
    music_df.loc[8397, ['thedate', 'mics_etc', 'date_format']] = ['1988-04-22', '_Muenster_14', 'byhand']
    music_df.loc[8597, ['thedate', 'mics_etc', 'date_format']] = ['2016-01-30', '-01-Intro-Edit', 'byhand']
    music_df.loc[8751, ['thedate', 'mics_etc', 'date_format']] = ['2008-07-27', '- Early 1', 'byhand']
    music_df.loc[8762, ['thedate', 'mics_etc', 'date_format']] = ['1976-01-01', ' _ 05-06d1t01', 'byhand']
    music_df.loc[8963, ['thedate', 'mics_etc', 'date_format']] = ['1988-11-03', '_munich_track_01', 'byhand']
    music_df.loc[8976, ['thedate', 'mics_etc', 'date_format']] = ['2004-06-11', '01', 'byhand']
    music_df.loc[8990, ['thedate', 'mics_etc', 'date_format']] = ['2012-07-26', '-01-Intro', 'byhand']
    music_df.loc[9017, ['thedate', 'mics_etc', 'date_format']] = ['1973-05-03', '03', 'byhand']
    music_df.loc[9101, ['thedate', 'mics_etc', 'date_format']] = ['1985-03-15', '_IW_D1T01', 'byhand']
    music_df.loc[9217, ['thedate', 'mics_etc', 'date_format']] = ['1989-07-29', ' D1 - Track 01', 'byhand']
    music_df.loc[9278, ['thedate', 'mics_etc', 'date_format']] = ['2009-11-13', ' Marin Civic Center 01', 'byhand']
    music_df.loc[9440, ['thedate', 'mics_etc', 'date_format']] = ['1988-10-21', '_nuernberg_track_01', 'byhand']
    music_df.loc[9462, ['thedate', 'mics_etc', 'date_format']] = ['2006-06-03', '_D2_T08-Pay Me My Money Down', 'byhand']
    music_df.loc[9519, ['thedate', 'mics_etc', 'date_format']] = ['2008-11-20', '_TM_01', 'byhand']
    music_df.loc[9569, ['thedate', 'mics_etc', 'date_format']] = ['1986-03-06', '_Rush_tSSVol1_D1T01', 'byhand']
    music_df.loc[9819, ['thedate', 'mics_etc', 'date_format']] = ['2008-06-09', '_Ostrava_TM_D1_01', 'byhand']
    music_df.loc[9866, ['thedate', 'mics_etc', 'date_format']] = ['2018-02-06', '_hannover_track_13', 'byhand']    
    music_df.loc[10070, ['thedate', 'mics_etc', 'date_format']] = ['1993-02-14', '_D1T01_Road Expense', 'byhand']    
    music_df.loc[10382, ['thedate', 'mics_etc', 'date_format']] = ['2018-02-26', '_hannover_track_04', 'byhand']
    music_df.loc[10390, ['thedate', 'mics_etc', 'date_format']] = ['2017-11-26', '_ludwigshafen_matrix_track_22', 'byhand']
    music_df.loc[10413, ['thedate', 'mics_etc', 'date_format']] = ['1980-03-04', '_ny_matrix_track_05', 'byhand']    
    music_df.loc[10422, ['thedate', 'mics_etc', 'date_format']] = ['2008-06-11', '_Salzburg_TM_D1_01', 'byhand']    
    music_df.loc[10507, ['thedate', 'mics_etc', 'date_format']] = ['2008-06-10', '_Vienna_TM_D1_01', 'byhand'] 
    music_df.loc[11248, ['thedate', 'mics_etc', 'date_format']] = ['1981-05-07', ' New York NY', 'byhand']
    music_df.loc[11659, ['thedate', 'mics_etc', 'date_format']] = ['2014-10-04', '.aud.mk4.banjostage-hsb-sf.flac16 10', 'byhand']
    music_df.loc[11660, ['thedate', 'mics_etc', 'date_format']] = ['2014-05-25', '.aud.mk4.santacruzbluesfestival.flac16 8', 'byhand']
    music_df.loc[11662, ['thedate', 'mics_etc', 'date_format']] = ['2015-05-23', '.aud.mk4.santa_cruz_american_music_festival 9', 'byhand']
    music_df.loc[11664, ['thedate', 'mics_etc', 'date_format']] = ['2015-10-04', '.aud.mk4.arrowstage-hsb-sf.flac16 10', 'byhand']    
    music_df.loc[11665, ['thedate', 'mics_etc', 'date_format']] = ['2014-10-03', '.aud.mk4.arrowstage-hsb-sf.flac16 06', 'byhand']
    music_df.loc[11667, ['thedate', 'mics_etc', 'date_format']] = ['2015-05-23', '.aud.mk4.santa_cruz_american_music_festival 17', 'byhand']
    music_df.loc[11668, ['thedate', 'mics_etc', 'date_format']] = ['2015-05-23', '.aud.mk4.santa_cruz_american_music_festival 9', 'byhand']
    music_df.loc[11689, ['thedate', 'mics_etc', 'date_format']] = ['2015-10-04', '.aud.mk4.towersofgoldstage-hsb-sf.flac16 09', 'byhand']
    music_df.loc[11691, ['thedate', 'mics_etc', 'date_format']] = ['2015-10-03', '.aud.mk4.arrowstage-hsb-sf.flac16 09', 'byhand']
    music_df.loc[11700, ['thedate', 'mics_etc', 'date_format']] = ['2014-05-05', '.aud.mk4.santacruzbluesfestival.flac16 28', 'byhand']   
    music_df.loc[11701, ['thedate', 'mics_etc', 'date_format']] = ['2014-06-21', '.aud.mk4.alamedacountyfair 20', 'byhand']
    music_df.loc[11705, ['thedate', 'mics_etc', 'date_format']] = ['2015-05-23', '.aud.mk4.santa_cruz_american_music_festival 11', 'byhand']
    music_df.loc[11718, ['thedate', 'mics_etc', 'date_format']] = ['2014-10-03', '.aud.mk4.banjostage-hsb-sf.flac 21', 'byhand']
    music_df.loc[11910, ['thedate', 'mics_etc', 'date_format']] = ['2017-07-21', '.05 oops! - \'greatest composition yet\' - Phish - Madison Square Garden, NY  set 1 eAT0801', 'byhand']
    music_df.loc[11911, ['thedate', 'mics_etc', 'date_format']] = ['2017-03-01', '06 Goin\' Home (tease) - Tom San Filippo - Toomey\'s Tavern On The Crick, Amityville, NY  set 1 eA170301_0101', 'byhand']
    music_df.loc[11918, ['thedate', 'mics_etc', 'date_format']] = ['2017-03-22', '01 tuning - TSF - Toomey\'s Tavern On The Crik, Amityville, NY  set 1 eA170322_0101', 'byhand']    
    music_df.loc[11919, ['thedate', 'mics_etc', 'date_format']] = ['2017-02-08', '15 tuning - Tom San Filippo - Toomey\'s Tavern On The Crik, Amityville, NY  set 2 eA170208_0201', 'byhand']        
    music_df.loc[11969, ['thedate', 'mics_etc', 'date_format']] = ['2017-04-19', '01 tuning - Tom San Filippo - Toomey\'s Tavern On The Crik, Amityville, NY  set 1 eA170419_0101', 'byhand']        
    music_df.loc[11920, ['thedate', 'mics_etc', 'date_format']] = ['2017-03-22', '01 tuning - TSF - Toomey\'s Tavern On The Crik, Amityville, NY  set 1 eA170322_0101', 'byhand']        
    music_df.loc[12236, ['thedate', 'mics_etc', 'date_format']] = ['2017-07-22', ' Opening for The Who01', 'byhand']        
    music_df.loc[12238, ['thedate', 'mics_etc', 'date_format']] = ['2017-07-22', 'MK-601', 'byhand']        
    music_df.loc[12332, ['thedate', 'mics_etc', 'date_format']] = ['2017-10-21', 'd01t01', 'byhand']        
    music_df.loc[12469, ['thedate', 'mics_etc', 'date_format']] = ['2017-09-30', 't01', 'byhand']
    music_df.loc[12635, ['thedate', 'mics_etc', 'date_format']] = ['2017-05-04', '_00104', 'byhand']
    music_df.loc[12870, ['thedate', 'mics_etc', 'date_format']] = ['1986-07-04', '.s1t01', 'byhand']    
    music_df.loc[12929, ['thedate', 'mics_etc', 'date_format']] = ['1998-08-15', 'd1t1', 'byhand']
    music_df.loc[12930, ['thedate', 'mics_etc', 'date_format']] = ['1998-08-16', 'd1t1', 'byhand']
    music_df.loc[12946, ['thedate', 'mics_etc', 'date_format']] = ['1988-09-15', '.s1t01', 'byhand']
    music_df.loc[14304, ['thedate', 'mics_etc', 'date_format']] = ['2018-01-15', 's1t01', 'byhand']
    music_df.loc[14843, ['thedate', 'mics_etc', 'date_format']] = ['2018-02-10', ' set 2 t09', 'byhand']
    music_df.loc[14875, ['thedate', 'mics_etc', 'date_format']] = ['2018-02-09', '_SonyPCM-D50_24x96_s01t01', 'byhand']
    music_df.loc[14881, ['thedate', 'mics_etc', 'date_format']] = ['2018-02-12', '-KM184_chris_pavicich-S02T11-Peel', 'byhand']
    music_df.loc[15002, ['thedate', 'mics_etc', 'date_format']] = ['1986-07-04', '.s1t01', 'byhand']
    music_df.loc[15031, ['thedate', 'mics_etc', 'date_format']] = ['1986-07-04', '.s1t01', 'byhand']
    music_df.loc[15995, ['thedate', 'mics_etc', 'date_format']] = ['1972-05-07', '24-01 Dark Star', 'byhand']
    music_df.loc[15996, ['thedate', 'mics_etc', 'date_format']] = ['1972-04-07', '19-01 Playing In The Band', 'byhand']
    music_df.loc[15997, ['thedate', 'mics_etc', 'date_format']] = ['1970-02-23', '13-01 Little Sadie', 'byhand']
    music_df.loc[15998, ['thedate', 'mics_etc', 'date_format']] = ['1979-11-30', '1-01 Passenger', 'byhand']
    music_df.loc[16066, ['thedate', 'mics_etc', 'date_format']] = ['1995-07-29', '1/RDANKO NYC  EARLY08', 'byhand']
    music_df.loc[16620, ['thedate', 'mics_etc', 'date_format']] = ['1988-09-15', '.s1t01', 'byhand']
    music_df.loc[17108, ['thedate', 'mics_etc', 'date_format']] = ['1986-07-04', '.s1t01', 'byhand']
    music_df.loc[17179, ['thedate', 'mics_etc', 'date_format']] = ['2018-07-16', '_S101', 'byhand']
    music_df.loc[17501, ['thedate', 'mics_etc', 'date_format']] = ['2017-08-06', 'd2t01', 'byhand']
    music_df.loc[17797, ['thedate', 'mics_etc', 'date_format']] = ['1975-07-07', '_01 Trenchtown Rock', 'byhand']
    music_df.loc[17821, ['thedate', 'mics_etc', 'date_format']] = ['1983-04-26', 'd1t01shakedown', 'byhand']
    music_df.loc[18040, ['thedate', 'mics_etc', 'date_format']] = ['1989-12-15', '_D101', 'byhand']
    music_df.loc[18098, ['thedate', 'mics_etc', 'date_format']] = ['1993-02-14', '_D1T01_Road Expense', 'byhand']
    music_df.loc[18356, ['thedate', 'mics_etc', 'date_format']] = ['1977-06-02', 'D01T01', 'byhand']
    music_df.loc[18359, ['thedate', 'mics_etc', 'date_format']] = ['2000-03-15', '-csheb-d1t01', 'byhand']
    music_df.loc[18386, ['thedate', 'mics_etc', 'date_format']] = ['2000-03-16', '-csheb-d1t01', 'byhand']    
    music_df.loc[18633, ['thedate', 'mics_etc', 'date_format']] = ['2000-06-18', 'd1t01', 'byhand']
    music_df.loc[18752, ['thedate', 'mics_etc', 'date_format']] = ['1979-09-12', '_Rush_UtA79M_D1T01', 'byhand']
    music_df.loc[18890, ['thedate', 'mics_etc', 'date_format']] = ['2009-05-31', 't01', 'byhand']
    music_df.loc[19348, ['thedate', 'mics_etc', 'date_format']] = ['1977-02-15', '1. BMW - Don Kirshner\'s Rock Concert -  - No Woman, No Cry', 'byhand']
    music_df.loc[20128, ['thedate', 'mics_etc', 'date_format']] = ['2000-10-05', 'd1t01', 'byhand']
    music_df.loc[20129, ['thedate', 'mics_etc', 'date_format']] = ['2000-11-12', 'd1t01', 'byhand']
    music_df.loc[20361, ['thedate', 'mics_etc', 'date_format']] = ['1971-08-21', '1-0...d1t02', 'byhand']
    music_df.loc[20605, ['thedate', 'mics_etc', 'date_format']] = ['1986-08-12', '_01', 'byhand']
    music_df.loc[20606, ['thedate', 'mics_etc', 'date_format']] = ['1986-11-13', '_01', 'byhand']
    music_df.loc[20667, ['thedate', 'mics_etc', 'date_format']] = ['1971-04-27', ' 202', 'byhand']
    music_df.loc[20766, ['thedate', 'mics_etc', 'date_format']] = ['2016-10-01', '_t01', 'byhand']
    music_df.loc[20770, ['thedate', 'mics_etc', 'date_format']] = ['1988-06-15', '1 Artist..Metallica ', 'byhand']
    music_df.loc[20776, ['thedate', 'mics_etc', 'date_format']] = ['1985-03-17', '_01', 'byhand']
    music_df.loc[20779, ['thedate', 'mics_etc', 'date_format']] = ['2000-03-25', 'ms01', 'byhand']
    music_df.loc[20978, ['thedate', 'mics_etc', 'date_format']] = ['1986-08-14', '01 Van Halen_Live__-_Intro', 'byhand']

    music_df.loc[21246, ['thedate', 'mics_etc', 'date_format']] = ['1980-04-15', '_1644_t01', 'byhand']
    music_df.loc[21315, ['thedate', 'mics_etc', 'date_format']] = ['1980-04-15', '_1644_t01', 'byhand']
    



    music_df.loc[274, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1992, 1, np.nan, '-intro-tawn_mastrey', 'partsbyhand']
    music_df.loc[1827, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1986, np.nan, np.nan, 't01', 'partsbyhand']
    music_df.loc[1849, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1992, 4, np.nan, 't01', 'partsbyhand']
    music_df.loc[2176, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [np.nan, 10, 23, '01 quartet no 1', 'partsbyhand']
    music_df.loc[2930, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1975, np.nan, np.nan, '_Hamilton_000', 'partsbyhand']
    music_df.loc[3203, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1976, np.nan, np.nan, '_sbd_000', 'partsbyhand']
    music_df.loc[3251, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [2017, np.nan, np.nan, '01 jazzfest belin 30-31', 'partsbyhand']
    music_df.loc[3580, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1980, 4, np.nan, ' Saga Heilbronn FM01', 'partsbyhand']
    music_df.loc[4039, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1980, 10, np.nan, '_T10', 'partsbyhand']
    music_df.loc[4926, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [2008, 1, np.nan, ' 19th Nervous Breakdown', 'partsbyhand']
    music_df.loc[7328, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1990, 9, np.nan, '_disc_1_t01', 'partsbyhand']
    music_df.loc[11925, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1986, 10, np.nan, 'xCulpepperT01', 'partsbyhand']
    music_df.loc[12007, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [np.nan, np.nan, np.nan, '2.1.3 ~ normalization, track & ', 'partsbyhand']
    music_df.loc[16266, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1968, np.nan, np.nan, 'MidnightHour', 'partsbyhand']
    music_df.loc[16831, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1984, np.nan, np.nan, '.011885.interview', 'partsbyhand']
    music_df.loc[17638, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1966, 11, np.nan, '.FM.interview', 'partsbyhand']
    music_df.loc[18059, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1969, 6, np.nan, '.aud.tzuriel.sbeok', 'partsbyhand']
    music_df.loc[18222, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1970, np.nan, np.nan, '.007319.outakes', 'partsbyhand']
    music_df.loc[18788, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [1988, np.nan, np.nan, '01_ww1_-24-intro', 'partsbyhand']
    music_df.loc[19129, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [np.nan, np.nan, np.nan, '01 9.28 Am', 'partsbyhand']
    music_df.loc[19130, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [np.nan, np.nan, np.nan, '01 9.28 AM', 'partsbyhand']
    music_df.loc[19500, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [np.nan, np.nan, np.nan, '01 9.28 AM', 'partsbyhand']
    music_df.loc[19501, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [np.nan, np.nan, np.nan, '01 9.28 AM', 'partsbyhand']

    music_df.loc[20489, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [np.nan, np.nan, np.nan, '01 9.28 A.M', 'partsbyhand']
    music_df.loc[21158, ['theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = [2004, np.nan, np.nan, '_CD1_01_WolfgangNiedecken_TraditionalSong', 'partsbyhand']

    return df
print('ok')

In [ ]:

music_df.loc[(music_df['postbandstuff'].apply(lambda x: re.search(yyyyfinder, x)).notnull()), 'date_format'] = 'yyyy-mm-dd'
music_df.loc[(music_df['date_format'] == 'otros') & (music_df['postbandstuff'].apply(lambda x: re.search(yyfinder, x)).notnull()), 'date_format'] = 'yy-mm-dd'

music_df.loc[
    (music_df['thedate'].isnull()) & 
    (music_df['postbandstuff'].apply(lambda x: re.search(date1, x)).notnull())
    , 'thedate'] = pandas.to_datetime(music_df['postbandstuff'][(music_df['thedate'].isnull()) & (music_df['postbandstuff'].apply(lambda x: re.search(date1, x)).notnull())
                                                               ].apply(lambda x: x[re.search(date1, x).start():re.search(date1, x).end()]), errors='coerce')
music_df.loc[(music_df['date_format'] == 'otros') & (music_df['thedate'].notnull()), 'date_format'] = '2 or 4 digit year, separator, 1 or 2 digit month, separator, 1 or 2 digit day'




print(music_df.groupby('date_format').size())

music_df[['thedate', 'date_format']][music_df['date_format'] == 'otros'].sample(10)


In [ ]:
music_df = music_df_bk

# Fix some dates by hand
music_df.loc[['567662', '567649', '567750'], ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-23'), 'byhand']
music_df.loc[['565762', '565781'], ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-03'), 'byhand']
music_df.loc[['565838', '126834'], ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-02'), 'byhand']
music_df.loc[['565838', '126834'], ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-02'), 'byhand']
music_df.loc['566023', ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-08'), 'byhand']
music_df.loc['566144', ['thedate', 'date_format']
            ] = [pandas.to_datetime('1982-11-05'), 'byhand']
music_df.loc['566144', ['thedate', 'date_format']
            ] = [pandas.to_datetime('1982-11-05'), 'byhand']
music_df.loc['566231', ['thedate', 'date_format']
            ] = [pandas.to_datetime('1988-02-29'), 'byhand']
music_df.loc['566526', ['thedate', 'date_format']
            ] = [pandas.to_datetime('1978-11-13'), 'byhand']
music_df.loc['567038', ['thedate', 'date_format']
            ] = [pandas.to_datetime('2003-09-23'), 'byhand']
music_df.loc['567500', ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-26'), 'byhand']
music_df.loc[['567549', '127174'], ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-27'), 'byhand']
music_df.loc['566758', ['theyear', 'themonth', 'date_format']
            ] = [1991, 7, 'byhand']

# Discovered through regex search for [\d]{6}
music_df.loc['566210', ['thedate', 'date_format']] = [pandas.to_datetime('2016-01-08'), 'byhand']
music_df.loc['566223', ['thedate', 'date_format']] = [pandas.to_datetime('1988-07-09'), 'byhand']
music_df.loc['566252', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-09'), 'byhand']
music_df.loc['567003', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-17'), 'byhand']
music_df.loc['567140', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-20'), 'byhand']
music_df.loc['567671', ['thedate', 'date_format']] = [pandas.to_datetime('1989-11-19'), 'byhand']
music_df.loc['567872', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-30'), 'byhand']

# Discovered by looking through each value
music_df.loc['567291', ['thedate', 'date_format']] = [pandas.to_datetime('2000-02-25'), 'byhand']
music_df.loc['567291', ['themonth', 'theday', 'date_format']] = [7, 4, 'byhand']
music_df.loc['565561', ['thedate', 'date_format']] = [pandas.to_datetime('2011-09-06'), 'byhand']
music_df.loc['567156', ['thedate', 'date_format']] = [pandas.to_datetime('2014-04-26'), 'byhand']
music_df.loc['566417', ['thedate', 'date_format']] = [pandas.to_datetime('2013-03-08'), 'byhand']
music_df.loc['565929', ['thedate', 'date_format']] = [pandas.to_datetime('2006-03-01'), 'byhand']
music_df.loc['567639', ['thedate', 'date_format']] = [pandas.to_datetime('1975-01-03'), 'byhand']
music_df.loc['587454', ['thedate', 'date_format']] = [pandas.to_datetime('2016-01-30'), 'byhand']
music_df.loc['566234', ['thedate', 'date_format']] = [pandas.to_datetime('1979-11-18'), 'byhand']
music_df.loc['567376', ['thedate', 'date_format']] = [pandas.to_datetime('1977-08-01'), 'byhand']
music_df.loc['587339', ['thedate', 'date_format']] = [pandas.to_datetime('2016-02-19'), 'byhand']
music_df.loc['567790', ['thedate', 'date_format']] = [pandas.to_datetime('2017-07-26'), 'byhand']
music_df.loc['567790', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-22'), 'byhand']
music_df.loc['566157', ['thedate', 'date_format']] = [pandas.to_datetime('1979-11-24'), 'byhand']
music_df.loc['567572', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-22'), 'byhand']
music_df.loc['565793', ['thedate', 'date_format']] = [pandas.to_datetime('1981-06-27'), 'byhand']
music_df.loc['587337', ['thedate', 'date_format']] = [pandas.to_datetime('2016-06-03'), 'byhand']
music_df.loc['587588', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-10'), 'byhand']
music_df.loc['587640', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-08'), 'byhand']
music_df.loc['587572', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-09'), 'byhand']
music_df.loc['126880', ['thedate', 'date_format']] = [pandas.to_datetime('1988-10-01'), 'byhand']
music_df.loc['567487', ['themonth', 'theday', 'date_format']] = [8, 14, 'byhand']
music_df.loc['566266', ['themonth', 'theday', 'date_format']] = [11, 24, 'byhand']
music_df.loc['565981', ['themonth', 'theday', 'date_format']] = [8, 29, 'byhand']
music_df.loc['567478', ['theyear', 'themonth', 'date_format']] = [2000, 6, 'byhand']
music_df.loc['567323', ['theyear', 'date_format']] = [1983, 'byhand']
music_df.loc['567526', ['themonth', 'theday', 'date_format']] = [10, 13, 'byhand']
music_df.loc['566219', ['theyear', 'date_format']] = [1990, 'byhand']
music_df.loc['565891', ['theyear', 'date_format']] = [1989, 'byhand']
music_df.loc['566208', ['theyear', 'themonth', 'date_format']] = [1972, 1, 'byhand']
music_df.loc['e059e8d39da4b75145b1e434b0569df9c8f59367', ['theyear', 'date_format']] = [1995, 'byhand']
music_df.loc['567222', ['themonth', 'theday', 'date_format']] = [2, 11, 'byhand']
music_df.loc['566756', ['theyear', 'themonth', 'date_format']] = [2011, 1, 'byhand']
music_df.loc['566756', ['theyear', 'themonth', 'date_format']] = [2016, 1, 'byhand']
music_df.loc['566756', ['theyear', 'themonth', 'date_format']] = [1980, 7, 'byhand']


music_df.loc[(music_df['theyear'].isnull()) & (music_df['thedate'].notnull())
    , 'theyear'] = music_df['thedate'][(music_df['theyear'].isnull()) & (music_df['thedate'].notnull())
                                      ].apply(lambda x: x.year)
music_df.loc[(music_df['themonth'].isnull()) & (music_df['thedate'].notnull())
    , 'themonth'] = music_df['thedate'][(music_df['themonth'].isnull()) & (music_df['thedate'].notnull())
                                      ].apply(lambda x: x.month)
music_df.loc[(music_df['theday'].isnull()) & (music_df['thedate'].notnull())
    , 'theday'] = music_df['thedate'][(music_df['theday'].isnull()) & (music_df['thedate'].notnull())
                                      ].apply(lambda x: x.day)

music_df.loc[music_df['date_format'] == 'otros', 'date_format'] = 'nodate'
music_df_bk = music_df

music_df.groupby('date_format').size()

In [ ]:
seps = re.compile('[\s\-\.\/\\_]')

music_df = music_df_bk

music_df['date_right_after_band'] = False
music_df.loc[
    (music_df['band_abbrev_autoparsed'] != '') &
    ((music_df['band_abbrev_autoparsed'].apply(lambda x: re.search(seps, x[-1:])).isnull())) &
    (music_df['postbandstuff'] != '') &
    (music_df['postbandstuff'].apply(lambda x: re.search('\d', x[0:])).notnull()) &
    (music_df['thedate'].notnull() | music_df['theyear'].notnull() | music_df['themonth'].notnull() | music_df['theday'].notnull())
    ,'date_right_after_band'] = True

music_df_bk = music_df
music_df.groupby('date_right_after_band').size()

In [ ]:
#music_df.loc[(music_df[''])]
music_df = music_df_bk

# Full 4 points for: 
#    * band abbrev is canonical or assigned (correct but not in etree's list)
#    * date directly follows band with no space between
#    * date is correctly formatted as yyyy-mm-dd

music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'assigned'])) & 
    (music_df['date_format'] == 'yyyy-mm-dd') & 
    (music_df['date_right_after_band']), 'score'] = 4

# Score goes down to 3 if...
#    date is formatted as yy-mm-dd, OR
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'assigned'])) & 
    (music_df['date_format'] == 'yy-mm-dd') & 
    (music_df['date_right_after_band']), 'score'] = 3

#    date does not directly follow band, OR
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'assigned'])) & 
    (music_df['date_format'] == 'yyyy-mm-dd') & 
    (~music_df['date_right_after_band']), 'score'] = 3

#    they use a variant abbreviation
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['variant'])) & 
    (music_df['date_format'] == 'yyyy-mm-dd') & 
    (music_df['date_right_after_band']), 'score'] = 4

# Score goes down to 2 if...
#    date is present and readable but formatted strangely, OR
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'assigned', 'variant'])) & 
    (music_df['date_format'].apply(lambda x: 
                                   x in ['2 or 4 digit year, separator, 1 or 2 digit month, separator, 1 or 2 digit day', 'byhand']))\
    , 'score'] = 2

#    if the band has a correct abbreviation but they used an incorrect one
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['incorrect'])) & 
    (music_df['date_format'].apply(lambda x: 
                                   x in ['yyyy-mm-dd', 'yy-mm-dd']))
    , 'score'] = 2

# Score = 0 if there is no band AND no date
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['blank', 'notband'])) & 
    (music_df['date_format'] == 'nodate'), 'score'] = 0

# OTHERWISE, score = 1
music_df.loc[music_df['score'] == -9999, 'score'] = 1

music_df_bk = music_df
#music_df.groupby('score').size()
#music_df[['filename', 'source', 'filetype', 'band_abbrev', 'abbrev_type', 'band', 'thedate', 'date_format', 'date_right_after_band', 'score']][
music_df['filename'][music_df['score'] ==  0].sample(1)

In [ ]:
music_df[['filename', 'source', 'filetype', 'band_abbrev', 'abbrev_type', 'band', 'thedate', 'date_format', 'date_right_after_band', 'score']
        ].sort_values(by='score', ascending=False).to_csv('outmusic.csv')
print('ok')

In [ ]:
music_df = music_df_bk

music_df['band'] = music_df['band'].fillna('blank')

small_sample_cutoff_value = 5

stats_allbands_df = pandas.DataFrame(data=music_df['band'].drop_duplicates().sort_values().values, columns=['band'])
stats_allbands_df = stats_allbands_df.set_index('band')

stats_allbands_df['jamband'] = 'no'
jambandlist = ['Grateful Dead', 'The Other Ones', 'The Dead']
jambandlist += ['Phil Lesh & Friends', 'Ratdog', 'Jerry Garcia Band']
jambandlist += ['Jerry Garcia', 'Bruce Hornsby', 'Phish', 'Trey Anastasio Band']
jambandlist += ['Oysterhead']
jambandlist += ['Georgia Avenue & Ratdog', 'Dead & Company']
jambandlist += ['Trey Anastasio (Band)']

jambandmaybelist = ['The Big Wu', 'Blues Traveler', 'Built to Spill']
jambandmaybelist += ['Dark Star Orchestra', 'Dave Matthews Band', 'Dave Matthews']
jambandmaybelist += ['Dave Matthews & Tim Reynolds', 'Drive-By Truckers']
jambandmaybelist += ['Gov\'t Mule', 'Guster', 'Hot Tuna']
jambandmaybelist += ['Les Claypool\'s Fearless Flying Frog Brigade']
jambandmaybelist += ['Col. Claypool\'s Bucket of Bernie Brains']
jambandmaybelist += ['Medeski Martin & Wood', 'moe', 'String Cheese Incident']
jambandmaybelist += ['Umphrey\'s McGee', 'Ween', 'Widespread Panic']
jambandmaybelist += ['Yonder Mountain String Band']
jambandmaybelist += ['Dave Matthews (solo)', 'Tim Reynolds']

stats_allbands_df.loc[stats_allbands_df.index.isin(jambandlist), 'jamband'] = 'yes'
stats_allbands_df.loc[stats_allbands_df.index.isin(jambandmaybelist), 'jamband'] = 'maybe'

stats_allbands_df['count'] = 0
stats_allbands_df['count_zero'] = 0
stats_allbands_df['count_one'] = 0
stats_allbands_df['count_two'] = 0
stats_allbands_df['count_three'] = 0
stats_allbands_df['count_four'] = 0
stats_allbands_df['median'] = 0
stats_allbands_df['mean'] = 0
stats_allbands_df['stdev'] = 0

for thisband, thisrow in stats_allbands_df.iterrows():
    stats_allbands_df.loc[thisband, 'count'] = music_df['score'][music_df['band'] == thisband].count()
    stats_allbands_df.loc[thisband, 'count_zero'] = music_df['score'][(music_df['band'] == thisband) & (music_df['score'] == 0)].count()
    stats_allbands_df.loc[thisband, 'count_one'] = music_df['score'][(music_df['band'] == thisband) & (music_df['score'] == 1)].count()
    stats_allbands_df.loc[thisband, 'count_two'] = music_df['score'][(music_df['band'] == thisband) & (music_df['score'] == 2)].count()
    stats_allbands_df.loc[thisband, 'count_three'] = music_df['score'][(music_df['band'] == thisband) & (music_df['score'] == 3)].count()
    stats_allbands_df.loc[thisband, 'count_four'] = music_df['score'][(music_df['band'] == thisband) & (music_df['score'] == 4)].count()
    stats_allbands_df.loc[thisband, 'median'] = music_df['score'][music_df['band'] == thisband].median()
    stats_allbands_df.loc[thisband, 'mean'] = music_df['score'][music_df['band'] == thisband].mean()
    stats_allbands_df.loc[thisband, 'stdev'] = music_df['score'][music_df['band'] == thisband].std()
    
#music_df['score'][(music_df['band'] == 'Grateful Dead')].count()
stats_df = stats_allbands_df[stats_allbands_df['count'] >= small_sample_cutoff_value]

misc_bands = stats_allbands_df[stats_allbands_df['count'] < small_sample_cutoff_value].index.values.tolist()
misc_count = music_df['score'][music_df['band'].isin(misc_bands)].count()
misc_count_zero = music_df['score'][(music_df['band'].isin(misc_bands)) & (music_df['score'] == 0)].count()
misc_count_one = music_df['score'][(music_df['band'].isin(misc_bands)) & (music_df['score'] == 1)].count()
misc_count_two = music_df['score'][(music_df['band'].isin(misc_bands)) & (music_df['score'] == 2)].count()
misc_count_three = music_df['score'][(music_df['band'].isin(misc_bands)) & (music_df['score'] == 3)].count()
misc_count_four = music_df['score'][(music_df['band'].isin(misc_bands)) & (music_df['score'] == 4)].count()
misc_median = music_df['score'][music_df['band'].isin(misc_bands)].median()
misc_mean = music_df['score'][music_df['band'].isin(misc_bands)].mean()
misc_stdev = music_df['score'][music_df['band'].isin(misc_bands)].std()
allotherbands_df = pandas.DataFrame(data = [['no', misc_count, misc_count_zero, misc_count_one, misc_count_two, misc_count_three, misc_count_four, misc_median, misc_mean, misc_stdev]], columns=stats_df.columns, index=['All other bands'])
stats_df = stats_df.append(allotherbands_df)

stats_df.sort_values('mean', ascending=False)

In [ ]:
music_df = music_df_bk
music_df = music_df.join(stats_allbands_df['jamband'], how='left', on='band')
music_df.groupby('jamband')['score'].mean()


from scipy.stats import ttest_ind

group1 = music_df['score'][music_df['jamband'] == 'yes']
group2 = music_df['score'][music_df['jamband'] == 'maybe']

t, p = ttest_ind(group1, group2)
print(p)